# Ecommerce Clickstream Transaction Data
- Pandas Manipulation Practice
- Typical Ecommerce Clickstream + Transaction Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("ecommerce_clickstream_transactions.csv")
data.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN


# Data Overview

In [3]:
data.shape

(74817, 7)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74817 entries, 0 to 74816
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   UserID     74817 non-null  int64  
 1   SessionID  74817 non-null  int64  
 2   Timestamp  74817 non-null  object 
 3   EventType  74817 non-null  object 
 4   ProductID  32113 non-null  object 
 5   Amount     10682 non-null  float64
 6   Outcome    10682 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.0+ MB


In [5]:
print("# of Users:", data.UserID.nunique())
print("Time Range:", data.Timestamp.min(), data.Timestamp.max())

# of Users: 1000
Time Range: 2024-01-01 00:01:35.004064 2024-07-24 10:13:04.983908


In [6]:
data.EventType.value_counts()

page_view       10819
add_to_cart     10735
product_view    10696
logout          10685
purchase        10682
click           10632
login           10568
Name: EventType, dtype: int64

In [7]:
print(data.ProductID.isna().mean())
data[~data.ProductID.isna()].EventType.value_counts()

0.5707793683253806


add_to_cart     10735
product_view    10696
purchase        10682
Name: EventType, dtype: int64

In [8]:
data.Outcome.value_counts()

purchase    10682
Name: Outcome, dtype: int64

In [9]:
data[~data.Outcome.isna()].EventType.value_counts()

purchase    10682
Name: EventType, dtype: int64

# Data Setup

In [10]:
data.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN


In [11]:
df = data.copy()
df.head(2)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN


# Pandas Questions + Answers

### Q: How many events does each user generate?
- size()

In [12]:
type(df.groupby("UserID").size())

pandas.core.series.Series

In [13]:
type(df.groupby("UserID").size().reset_index())

pandas.core.frame.DataFrame

In [14]:
df.groupby("UserID").size().rename("EventCount").reset_index().sort_values("EventCount", ascending=False)

,UserID,EventCount
978,979,94
779,780,92
184,185,90
911,912,89
509,510,89
...,...,...
382,383,61
152,153,61
516,517,60
726,727,59


### Q: DAU/ WAU/ MAU: How many active users per day/ week/ month?
- pd.to_datetime()
- df.set_index("Timestamp)
- pd.Grouper(freq="W"/"M")
- nunique()

In [15]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["Date"] = df["Timestamp"].dt.date
df["Hour"] = df["Timestamp"].dt.hour

In [16]:
df.groupby("Date")["UserID"].nunique().rename("DAU").reset_index().head()

,Date,DAU
0,2024-01-01,329
1,2024-01-02,306
2,2024-01-03,332
3,2024-01-04,298
4,2024-01-05,304


In [17]:
df.set_index("Timestamp").groupby(pd.Grouper(freq="W"))["UserID"].nunique().rename("WAU").reset_index().head()

,Timestamp,WAU
0,2024-01-07,928
1,2024-01-14,911
2,2024-01-21,918
3,2024-01-28,916
4,2024-02-04,936


In [18]:
df.set_index("Timestamp").groupby(pd.Grouper(freq="M"))["UserID"].nunique().rename("WAU").reset_index().head()

,Timestamp,WAU
0,2024-01-31,1000
1,2024-02-29,1000
2,2024-03-31,1000
3,2024-04-30,1000
4,2024-05-31,1000


### Q: Which products have the most unique product viewers?
- filter
- nunique()

In [19]:
df[df.EventType == "product_view"].groupby("ProductID")["UserID"].nunique().rename("UniqueViewers").reset_index().sort_values("UniqueViewers", ascending=False).head(10)

,ProductID,UniqueViewers
1839,prod_3587,9
3598,prod_6092,7
6181,prod_9806,7
533,prod_1750,6
2629,prod_4714,6
675,prod_1964,6
4883,prod_7938,6
946,prod_2346,6
1806,prod_3549,6
982,prod_2390,6


### Q: How many unique sessions does each user have?

- nunique()

In [20]:
data.groupby("UserID")["SessionID"].nunique().rename("UniqueSessions").reset_index().sort_values("UniqueSessions", ascending=False)

,UserID,UniqueSessions
0,1,10
671,672,10
658,659,10
659,660,10
660,661,10
...,...,...
338,339,10
339,340,10
340,341,10
341,342,10


### Q: For each session, what is the start time, end time, and duration?
- agg
- size
- nunique
- min, max, max-min

In [21]:
session_span = df.groupby("SessionID")["Timestamp"].agg(
    SessionStart="min", 
    SessionEnd="max",
    SessionDutation=lambda x: x.max()-x.min()).reset_index()

session_span.head()

,SessionID,SessionStart,SessionEnd,SessionDutation
0,1,2024-01-01 00:26:02.151406,2024-07-24 10:00:22.891611,205 days 09:34:20.740205
1,2,2024-01-01 00:06:24.553313,2024-07-24 09:52:10.035079,205 days 09:45:45.481766
2,3,2024-01-01 00:55:30.407369,2024-07-24 08:55:04.644344,205 days 07:59:34.236975
3,4,2024-01-01 00:02:24.971658,2024-07-24 09:24:53.575048,205 days 09:22:28.603390
4,5,2024-01-01 00:28:14.885796,2024-07-24 07:26:23.963896,205 days 06:58:09.078100


### Q: What are the Top EventTypes by Count?

- value_counts()

In [22]:
df["EventType"].value_counts().reset_index().rename(columns={"index":"EventType", "EventType":"Count"}).sort_values("Count", ascending=False)

,EventType,Count
0,page_view,10819
1,add_to_cart,10735
2,product_view,10696
3,logout,10685
4,purchase,10682
5,click,10632
6,login,10568


### Q: For `product_view`/ `add_to_cart` events, what are the top products by number of unique users?

- df.eq()
- groupby(colA)[colB].nunique()
- sort_values(ascending=False)

In [23]:
df.head(2)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22


In [24]:
# product_view events with unique users per product
df[df["EventType"].eq("product_view")].groupby("ProductID")["UserID"].nunique().rename("UniqueUsers").reset_index().sort_values("UniqueUsers", ascending=False).head(10)

,ProductID,UniqueUsers
1839,prod_3587,9
3598,prod_6092,7
6181,prod_9806,7
533,prod_1750,6
2629,prod_4714,6
675,prod_1964,6
4883,prod_7938,6
946,prod_2346,6
1806,prod_3549,6
982,prod_2390,6


In [25]:
# add_to_cart events with unique users per product
df[df["EventType"].eq("add_to_cart")].groupby("ProductID")["UserID"].nunique().rename("UniqueUsers").reset_index().sort_values("UniqueUsers", ascending=False).head(10)

,ProductID,UniqueUsers
3062,prod_5459,8
1950,prod_3830,6
4473,prod_7436,6
5983,prod_9590,6
3482,prod_6051,6
767,prod_2090,6
855,prod_2207,6
5571,prod_8979,6
4028,prod_6820,6
2936,prod_5253,6


### Q: For each product, what is add-to-cart conversion rate given product views?
- Rate = # of users that add the product to cart / # of users that view the product

In [26]:
views = df[df["EventType"]=="product_view"].groupby("ProductID")["UserID"].nunique().rename("UniqueViewers")
carts = df[df["EventType"]=="add_to_cart"].groupby("ProductID")["UserID"].nunique().rename("UniqueAddtoCarts")
conversion = pd.concat([views, carts], axis=1).fillna(0)
conversion.head()

,UniqueViewers,UniqueAddtoCarts
ProductID,,
prod_1001,1.0,1.0
prod_1002,1.0,1.0
prod_1004,1.0,0.0
prod_1005,2.0,1.0
prod_1006,1.0,2.0


In [27]:
conversion["CartRate"] = conversion["UniqueAddtoCarts"] / conversion["UniqueViewers"].replace(0, np.nan)
conversion.sort_values("CartRate", ascending=False).head(10)

# Issue: the conversion rate can be over 1
# Some users may add to cart without viewing the product
# Fix: consider only users who viewed the product first

,UniqueViewers,UniqueAddtoCarts,CartRate
ProductID,,,
prod_7436,1.0,6.0,6.0
prod_2090,1.0,6.0,6.0
prod_3830,1.0,6.0,6.0
prod_6556,1.0,5.0,5.0
prod_5449,1.0,5.0,5.0
prod_5132,1.0,5.0,5.0
prod_5396,1.0,5.0,5.0
prod_4371,1.0,5.0,5.0
prod_4093,1.0,5.0,5.0


In [28]:
x = df[df["EventType"].isin(["product_view", "add_to_cart"]) & df["ProductID"].notna()].copy()
x["Viewed"] = (x["EventType"] == "product_view").astype(int)
x["Carted"] = (x["EventType"] == "add_to_cart").astype(int)
x.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,Viewed,Carted
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,1,0
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN,2024-03-12,0,0,1
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,0,1
6,1,1,2024-02-01 05:13:26.944169,add_to_cart,prod_4965,NaN,NaN,2024-02-01,5,0,1
12,1,2,2024-04-20 05:07:46.892185,product_view,prod_2683,NaN,NaN,2024-04-20,5,1,0


In [29]:
user_prod = (x.groupby(["UserID", "ProductID"], as_index=False)[["Viewed", "Carted"]].max())
user_prod = user_prod[user_prod.Viewed == 1]
user_prod["UserConversion"] = user_prod["Carted"] / user_prod["Viewed"]
user_prod.head()

,UserID,ProductID,Viewed,Carted,UserConversion
2,1,prod_1392,1,0,0.0
5,1,prod_2683,1,0,0.0
8,1,prod_3548,1,0,0.0
9,1,prod_3790,1,0,0.0
14,1,prod_5109,1,0,0.0


In [30]:
user_prod.groupby("ProductID")["UserConversion"].mean().rename("ProductConversion").reset_index().sort_values("ProductConversion", ascending=False).head(10)   


,ProductID,ProductConversion
1127,prod_2588,1.000000
4239,prod_7025,1.000000
3868,prod_6465,1.000000
5936,prod_9441,1.000000
5034,prod_8147,0.500000
5267,prod_8482,0.500000
4469,prod_7356,0.500000
275,prod_1387,0.333333
1954,prod_3751,0.333333
2148,prod_4053,0.333333


### Q: How many new users per day, how many returning users per day?

In [31]:
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN,2024-03-12,0
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22


In [32]:
first_seen = df.groupby("UserID")["Timestamp"].min().rename("FirstSeen").reset_index()
first_seen.head()

,UserID,FirstSeen
0,1,2024-01-01 23:09:51.956825
1,2,2024-01-01 14:07:40.491141
2,3,2024-01-05 15:44:25.330449
3,4,2024-01-01 02:11:06.298369
4,5,2024-01-01 14:03:07.709158


In [33]:
df = df.merge(first_seen, on="UserID", how="left")
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825


In [34]:
df["FirstSeenUser"] = (df["Timestamp"].dt.date == df["FirstSeen"].dt.date).astype(int)
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0


In [35]:
x = df.groupby(["Date", "UserID"])["FirstSeenUser"].max().reset_index()
x.head()

,Date,UserID,FirstSeenUser
0,2024-01-01,1,1
1,2024-01-01,2,1
2,2024-01-01,4,1
3,2024-01-01,5,1
4,2024-01-01,9,1


In [36]:
y = x.groupby("Date")["FirstSeenUser"].sum().rename("NewUsers").reset_index()
y.head()

,Date,NewUsers
0,2024-01-01,329
1,2024-01-02,205
2,2024-01-03,165
3,2024-01-04,85
4,2024-01-05,89


### Q: Compute D1/ D7 retention based on user's first active day

- D1 retention (“Day 1 retention”) = the % of users who are active the day after their first active day.
- D7 retention = the % of users who are active 7 days after their first active day.

In [37]:
df["ActiveDays"] = (df["Date"] - df["FirstSeen"].dt.date).dt.days
x = df[df["ActiveDays"]==1].groupby("Date")["UserID"].nunique().rename("D1ActiveUsers").reset_index()
x.head(3)

,Date,D1ActiveUsers
0,2024-01-02,101
1,2024-01-03,64
2,2024-01-04,47


In [38]:
y = df.groupby(["Date"])["UserID"].nunique().rename("TotalActiveUsers").reset_index()
y.head(3)

,Date,TotalActiveUsers
0,2024-01-01,329
1,2024-01-02,306
2,2024-01-03,332


In [39]:
pd.merge(x, y, on="Date").assign(D1Retention=lambda x: x["D1ActiveUsers"] / x["TotalActiveUsers"]).head(10)

,Date,D1ActiveUsers,TotalActiveUsers,D1Retention
0,2024-01-02,101,306,0.330065
1,2024-01-03,64,332,0.192771
2,2024-01-04,47,298,0.157718
3,2024-01-05,20,304,0.065789
4,2024-01-06,22,296,0.074324
5,2024-01-07,9,307,0.029316
6,2024-01-08,7,296,0.023649
7,2024-01-09,12,294,0.040816
8,2024-01-10,4,308,0.012987
9,2024-01-11,6,310,0.019355


In [40]:
m = df[df["ActiveDays"]<=7].groupby("Date")["UserID"].nunique().rename("D7ActiveUsers").reset_index()
m.head(3)

,Date,D7ActiveUsers
0,2024-01-01,329
1,2024-01-02,306
2,2024-01-03,332


In [41]:
n = df.groupby(["Date"])["UserID"].nunique().rename("TotalActiveUsers").reset_index()
n.head(3)

,Date,TotalActiveUsers
0,2024-01-01,329
1,2024-01-02,306
2,2024-01-03,332


In [42]:
pd.merge(m, n, on="Date").assign(D7Retention=lambda x: x["D7ActiveUsers"] / x["TotalActiveUsers"]).head(20)

,Date,D7ActiveUsers,TotalActiveUsers,D7Retention
0,2024-01-01,329,329,1.000000
1,2024-01-02,306,306,1.000000
2,2024-01-03,332,332,1.000000
3,2024-01-04,298,298,1.000000
4,2024-01-05,304,304,1.000000
5,2024-01-06,296,296,1.000000
6,2024-01-07,307,307,1.000000
7,2024-01-08,296,296,1.000000
8,2024-01-09,205,294,0.697279
9,2024-01-10,127,308,0.412338


### Q: Revenue by day + MoM/ WoW growth (pct_change)

In [43]:
df.head(2)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64


In [44]:
df[df["EventType"] == "purchase"].groupby("Date")["Amount"].sum().rename("DailyAmount").reset_index().assign(
    MoM_Growth=lambda x: x["DailyAmount"].pct_change(periods=30),
    WoW_Growth=lambda x: x["DailyAmount"].pct_change(periods=7)
).head(10)

,Date,DailyAmount,MoM_Growth,WoW_Growth
0,2024-01-01,13135.520675,NaN,NaN
1,2024-01-02,16436.927283,NaN,NaN
2,2024-01-03,11209.661466,NaN,NaN
3,2024-01-04,12885.815729,NaN,NaN
4,2024-01-05,9051.293775,NaN,NaN
5,2024-01-06,10815.875769,NaN,NaN
6,2024-01-07,13386.786711,NaN,NaN
7,2024-01-08,10003.502586,NaN,-0.238439
8,2024-01-09,10985.269222,NaN,-0.331671
9,2024-01-10,13092.081570,NaN,0.167928


### Q: What is the average order value (AOV) by day?

In [45]:
df[(df.EventType == "purchase") & (~df.ProductID.isna())].groupby("Date")["Amount"].mean().rename("AOV").reset_index().head(10)

,Date,AOV
0,2024-01-01,268.071851
1,2024-01-02,298.853223
2,2024-01-03,249.103588
3,2024-01-04,238.626217
4,2024-01-05,226.282344
5,2024-01-06,240.352795
6,2024-01-07,243.396122
7,2024-01-08,227.352332
8,2024-01-09,244.117094
9,2024-01-10,247.020407


# Pandas Manipulation Method Summary 

### I/O and quick inspection

- pd.read_csv, pd.read_parquet, pd.read_json, pd.read_excel
- DataFrame.to_csv, to_parquet, to_json, to_excel
- head, tail, sample
- info, describe, dtypes
- value_counts
- unique, nunique
- isna, notna
- memory_usage

In [46]:
df.tail()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
74812,1000,10,2024-05-11 22:48:45.500117,purchase,prod_1238,186.208225,purchase,2024-05-11,22,2024-01-03 15:54:55.591340,0,129
74813,1000,10,2024-03-29 04:09:32.514318,logout,NaN,NaN,NaN,2024-03-29,4,2024-01-03 15:54:55.591340,0,86
74814,1000,10,2024-02-09 02:58:56.128697,login,NaN,NaN,NaN,2024-02-09,2,2024-01-03 15:54:55.591340,0,37
74815,1000,10,2024-04-30 16:19:48.002633,purchase,prod_2515,198.757849,purchase,2024-04-30,16,2024-01-03 15:54:55.591340,0,118
74816,1000,10,2024-04-01 02:19:29.148727,logout,NaN,NaN,NaN,2024-04-01,2,2024-01-03 15:54:55.591340,0,89


In [47]:
df.sample(5)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
61217,819,4,2024-01-06 10:43:54.840353,page_view,NaN,NaN,NaN,2024-01-06,10,2024-01-02 06:58:25.111407,0,4
53582,717,4,2024-07-09 01:43:30.836318,logout,NaN,NaN,NaN,2024-07-09,1,2024-01-02 11:58:34.354087,0,189
4027,54,3,2024-02-15 21:57:57.638741,page_view,NaN,NaN,NaN,2024-02-15,21,2024-01-08 01:55:14.267910,0,38
74809,1000,10,2024-01-12 15:56:53.525921,product_view,prod_4566,NaN,NaN,2024-01-12,15,2024-01-03 15:54:55.591340,0,9
34861,467,1,2024-04-22 16:35:37.827474,logout,NaN,NaN,NaN,2024-04-22,16,2024-01-06 03:32:04.040059,0,107


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74817 entries, 0 to 74816
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   UserID         74817 non-null  int64         
 1   SessionID      74817 non-null  int64         
 2   Timestamp      74817 non-null  datetime64[ns]
 3   EventType      74817 non-null  object        
 4   ProductID      32113 non-null  object        
 5   Amount         10682 non-null  float64       
 6   Outcome        10682 non-null  object        
 7   Date           74817 non-null  object        
 8   Hour           74817 non-null  int64         
 9   FirstSeen      74817 non-null  datetime64[ns]
 10  FirstSeenUser  74817 non-null  int64         
 11  ActiveDays     74817 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(5), object(4)
memory usage: 7.4+ MB


In [49]:
df.dtypes

UserID                    int64
SessionID                 int64
Timestamp        datetime64[ns]
EventType                object
ProductID                object
Amount                  float64
Outcome                  object
Date                     object
Hour                      int64
FirstSeen        datetime64[ns]
FirstSeenUser             int64
ActiveDays                int64
dtype: object

In [50]:
df.dtypes.value_counts()

int64             5
object            4
datetime64[ns]    2
float64           1
dtype: int64

In [51]:
df[df.Timestamp.isna()]

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays


In [52]:
df[df.Timestamp.notna()].sample(5)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
32401,434,4,2024-01-18 16:55:10.839709,add_to_cart,prod_2795,NaN,NaN,2024-01-18,16,2024-01-05 05:10:07.761771,0,13
46794,626,2,2024-04-11 15:56:32.133097,login,NaN,NaN,NaN,2024-04-11,15,2024-01-01 22:58:28.424339,0,101
39711,531,6,2024-02-10 18:51:32.437186,page_view,NaN,NaN,NaN,2024-02-10,18,2024-01-08 01:03:05.109867,0,33
42144,564,2,2024-04-09 05:07:09.039423,page_view,NaN,NaN,NaN,2024-04-09,5,2024-01-02 19:34:50.044893,0,98
40637,543,10,2024-07-12 21:07:59.686436,product_view,prod_4881,NaN,NaN,2024-07-12,21,2024-01-04 17:43:47.008283,0,190


In [53]:
df.memory_usage()

Index            598536
UserID           598536
SessionID        598536
Timestamp        598536
EventType        598536
ProductID        598536
Amount           598536
Outcome          598536
Date             598536
Hour             598536
FirstSeen        598536
FirstSeenUser    598536
ActiveDays       598536
dtype: int64

### Indexing, Filtering, Selection

- [] column selection
- .loc, .iloc, .at, .iat
- query
- filter (by labels/regex)
- isin
- between
- where, mask
- str.contains, str.startswith
- clip
- drop, dropna
- duplicated, drop_duplicates

In [54]:
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55


In [55]:
df.loc[10]

UserID                                    1
SessionID                                 2
Timestamp        2024-03-31 07:56:23.690975
EventType                            logout
ProductID                               NaN
Amount                                  NaN
Outcome                                 NaN
Date                             2024-03-31
Hour                                      7
FirstSeen        2024-01-01 23:09:51.956825
FirstSeenUser                             0
ActiveDays                               90
Name: 10, dtype: object

In [56]:
df.loc[10:15]

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
10,1,2,2024-03-31 07:56:23.690975,logout,NaN,NaN,NaN,2024-03-31,7,2024-01-01 23:09:51.956825,0,90
11,1,2,2024-05-16 22:00:13.041682,click,NaN,NaN,NaN,2024-05-16,22,2024-01-01 23:09:51.956825,0,136
12,1,2,2024-04-20 05:07:46.892185,product_view,prod_2683,NaN,NaN,2024-04-20,5,2024-01-01 23:09:51.956825,0,110
13,1,2,2024-01-30 21:47:38.829172,page_view,NaN,NaN,NaN,2024-01-30,21,2024-01-01 23:09:51.956825,0,29
14,1,2,2024-03-06 23:01:09.517819,page_view,NaN,NaN,NaN,2024-03-06,23,2024-01-01 23:09:51.956825,0,65
15,1,3,2024-03-15 05:22:41.094810,purchase,prod_5244,72.913619,purchase,2024-03-15,5,2024-01-01 23:09:51.956825,0,74


In [57]:
df.loc[10:15, ["UserID", "EventType", "Amount"]]

,UserID,EventType,Amount
10,1,logout,NaN
11,1,click,NaN
12,1,product_view,NaN
13,1,page_view,NaN
14,1,page_view,NaN
15,1,purchase,72.913619


In [58]:
df.iloc[10:15]

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
10,1,2,2024-03-31 07:56:23.690975,logout,NaN,NaN,NaN,2024-03-31,7,2024-01-01 23:09:51.956825,0,90
11,1,2,2024-05-16 22:00:13.041682,click,NaN,NaN,NaN,2024-05-16,22,2024-01-01 23:09:51.956825,0,136
12,1,2,2024-04-20 05:07:46.892185,product_view,prod_2683,NaN,NaN,2024-04-20,5,2024-01-01 23:09:51.956825,0,110
13,1,2,2024-01-30 21:47:38.829172,page_view,NaN,NaN,NaN,2024-01-30,21,2024-01-01 23:09:51.956825,0,29
14,1,2,2024-03-06 23:01:09.517819,page_view,NaN,NaN,NaN,2024-03-06,23,2024-01-01 23:09:51.956825,0,65


In [59]:
df.iloc[10:15, 1:4]

,SessionID,Timestamp,EventType
10,2,2024-03-31 07:56:23.690975,logout
11,2,2024-05-16 22:00:13.041682,click
12,2,2024-04-20 05:07:46.892185,product_view
13,2,2024-01-30 21:47:38.829172,page_view
14,2,2024-03-06 23:01:09.517819,page_view


In [60]:
df.iloc[10:15, [1, 2, 5]]

,SessionID,Timestamp,Amount
10,2,2024-03-31 07:56:23.690975,NaN
11,2,2024-05-16 22:00:13.041682,NaN
12,2,2024-04-20 05:07:46.892185,NaN
13,2,2024-01-30 21:47:38.829172,NaN
14,2,2024-03-06 23:01:09.517819,NaN


In [61]:
### df.at: use it when you need to access a single value for a row/column label pair
### val = df.at[row_label, column_label]

df.at[10, "UserID"]
df.at[3, "EventType"]

'add_to_cart'

In [62]:
### set values using df.at
df.at[3, "EventType"] = "AABBCCDD"
df.at[3, "EventType"]

'AABBCCDD'

In [63]:
### filter rows using SQL-like string expression, instead of writing boolean masks
df.query("EventType == 'AABBCCDD'").head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71


In [64]:
### df index filters
df[df.index.isin([10, 20, 30])]

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
10,1,2,2024-03-31 07:56:23.690975,logout,NaN,NaN,NaN,2024-03-31,7,2024-01-01 23:09:51.956825,0,90
20,1,3,2024-02-15 12:46:46.387330,add_to_cart,prod_9851,NaN,NaN,2024-02-15,12,2024-01-01 23:09:51.956825,0,45
30,1,4,2024-07-15 16:15:52.074487,purchase,prod_9886,7.677938,purchase,2024-07-15,16,2024-01-01 23:09:51.956825,0,196


In [65]:
### df column filters
df[df.EventType.isin(["AABBCCDD", "product_view"])].head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71
12,1,2,2024-04-20 05:07:46.892185,product_view,prod_2683,NaN,NaN,2024-04-20,5,2024-01-01 23:09:51.956825,0,110
18,1,3,2024-05-29 13:33:10.268585,product_view,prod_9640,NaN,NaN,2024-05-29,13,2024-01-01 23:09:51.956825,0,149
24,1,4,2024-06-10 00:08:42.715781,product_view,prod_5771,NaN,NaN,2024-06-10,0,2024-01-01 23:09:51.956825,0,161


In [66]:
### df date filters
df[df.Date.between(pd.to_datetime("2024-01-03").date(), pd.to_datetime("2024-01-07").date())]

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
29,1,4,2024-01-04 09:25:49.705624,add_to_cart,prod_1102,NaN,NaN,2024-01-04,9,2024-01-01 23:09:51.956825,0,3
36,1,5,2024-01-03 23:51:05.729189,logout,NaN,NaN,NaN,2024-01-03,23,2024-01-01 23:09:51.956825,0,2
65,1,8,2024-01-04 17:09:29.677060,click,NaN,NaN,NaN,2024-01-04,17,2024-01-01 23:09:51.956825,0,3
72,1,9,2024-01-06 04:33:39.275154,page_view,NaN,NaN,NaN,2024-01-06,4,2024-01-01 23:09:51.956825,0,5
88,2,1,2024-01-04 03:53:06.596449,logout,NaN,NaN,NaN,2024-01-04,3,2024-01-01 14:07:40.491141,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
74713,999,8,2024-01-05 03:50:16.283889,product_view,prod_9377,NaN,NaN,2024-01-05,3,2024-01-01 14:48:07.008651,0,4
74765,1000,4,2024-01-07 23:49:08.108428,product_view,prod_7708,NaN,NaN,2024-01-07,23,2024-01-03 15:54:55.591340,0,4
74783,1000,7,2024-01-06 10:18:56.069670,product_view,prod_8279,NaN,NaN,2024-01-06,10,2024-01-03 15:54:55.591340,0,3
74785,1000,7,2024-01-03 15:54:55.591340,add_to_cart,prod_3091,NaN,NaN,2024-01-03,15,2024-01-03 15:54:55.591340,1,0


In [67]:
### between
### mask = df["column_name"].between(lower_bound, upper_bound, inclusive="both"/"left"/"right"/"neither")
mask = df["Amount"].between(50, 100, inclusive="both")
df_amount_50_100 = df[mask].copy()
df_amount_50_100.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
15,1,3,2024-03-15 05:22:41.094810,purchase,prod_5244,72.913619,purchase,2024-03-15,5,2024-01-01 23:09:51.956825,0,74
102,2,3,2024-01-20 07:23:41.874270,purchase,prod_5291,92.790219,purchase,2024-01-20,7,2024-01-01 14:07:40.491141,0,19
172,3,4,2024-07-07 21:23:11.190419,purchase,prod_6789,76.285324,purchase,2024-07-07,21,2024-01-05 15:44:25.330449,0,184
465,6,10,2024-03-03 01:47:03.184839,purchase,prod_3829,58.667072,purchase,2024-03-03,1,2024-01-04 18:52:51.924379,0,59
525,7,8,2024-05-24 20:36:47.826509,purchase,prod_9259,54.070602,purchase,2024-05-24,20,2024-01-07 11:49:12.672559,0,138


In [68]:
### between
### mask = df["Date"].between(pd.to_datetime(timestr1), pd.to_datetime((timestr2)))
mask = df["Date"].between(pd.to_datetime("2024-01-01"), pd.to_datetime(("2024-01-03")))
df_date_20240101_20250103 = df[mask].copy()
df_date_20240101_20250103.head()

/Users/wenxinzhang/miniconda3/envs/py38/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
5,1,1,2024-01-01 23:09:51.956825,page_view,NaN,NaN,NaN,2024-01-01,23,2024-01-01 23:09:51.956825,1,0
32,1,4,2024-01-02 00:15:51.420238,add_to_cart,prod_6144,NaN,NaN,2024-01-02,0,2024-01-01 23:09:51.956825,0,1
36,1,5,2024-01-03 23:51:05.729189,logout,NaN,NaN,NaN,2024-01-03,23,2024-01-01 23:09:51.956825,0,2
97,2,3,2024-01-03 01:39:14.214806,click,NaN,NaN,NaN,2024-01-03,1,2024-01-01 14:07:40.491141,0,2
103,2,4,2024-01-01 14:07:40.491141,add_to_cart,prod_8372,NaN,NaN,2024-01-01,14,2024-01-01 14:07:40.491141,1,0


In [69]:
### filter rows where EventType contains "CART" (case insensitive)
df[df.EventType.str.contains("CART", case=False, na=False)].sample()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
44299,592,9,2024-01-31 01:45:22.219120,add_to_cart,prod_1224,NaN,NaN,2024-01-31,1,2024-01-01 19:04:36.946929,0,30


In [70]:
### filter rows where EventType contains "CART" (case sensitive)
df[df.EventType.str.contains("CART", case=True, na=False)]

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays


In [71]:
### filter rows where EventType starts with "add"
df[df.EventType.str.startswith("add", na=False)].sample()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays
57527,770,1,2024-02-23 05:46:08.343282,add_to_cart,prod_2566,NaN,NaN,2024-02-23,5,2024-01-03 09:32:49.615665,0,51


In [72]:
### mask: replace values where condition is True
mask = df["EventType"] == "page_view"
x = df["EventType"].mask(mask) # mask the page_view as NaN
x

0                 NaN
1                 NaN
2        product_view
3            AABBCCDD
4         add_to_cart
             ...     
74812        purchase
74813          logout
74814           login
74815        purchase
74816          logout
Name: EventType, Length: 74817, dtype: object

In [73]:
### mask: replace values where condition is True
mask = df["EventType"] == "page_view"
x = df["EventType"].mask(mask, other="PV") # mask the page_view as PV
x

0                  PV
1                  PV
2        product_view
3            AABBCCDD
4         add_to_cart
             ...     
74812        purchase
74813          logout
74814           login
74815        purchase
74816          logout
Name: EventType, Length: 74817, dtype: object

In [74]:
### mask: replace values where condition is True
mask = df["Amount"] < 0
x = df["Amount"].mask(mask) # mask all the negative values as NaN
x

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
74812    186.208225
74813           NaN
74814           NaN
74815    198.757849
74816           NaN
Name: Amount, Length: 74817, dtype: float64

In [75]:
### where: keep values where condition is True, replace others with NaN
mask = df["Amount"] >= 50
df["AmountKept"] = df["Amount"].where(mask)
df[["Amount", "AmountKept"]].sample(10)

,Amount,AmountKept
41835,338.100474,338.100474
55508,NaN,NaN
9612,NaN,NaN
24519,NaN,NaN
21933,NaN,NaN
60628,NaN,NaN
66575,NaN,NaN
51932,NaN,NaN
25467,106.302482,106.302482
16078,NaN,NaN


In [76]:
### where: keep values where condition is True, replace others with 0
mask = df["Amount"] >= 50
df["AmountKept"] = df["Amount"].where(mask, other=0)
df[["Amount", "AmountKept"]].sample(10)

,Amount,AmountKept
72280,NaN,0.000000
48554,NaN,0.000000
67347,NaN,0.000000
17523,NaN,0.000000
20032,467.532208,467.532208
4842,NaN,0.000000
12747,NaN,0.000000
5435,NaN,0.000000
673,NaN,0.000000
23125,NaN,0.000000


- s.mask(condition) = s.where(~condition)
- `mask` triggers replacement when condition is True
- `where` keeps where condition is True

In [77]:
### clip: cap/ floor values
### greater than upper bound -> set to upper bound
### less than lower bound -> set to lower bound 
### great for outliers/ winsorizing
df["AmountClipped"] = df["Amount"].clip(lower=0, upper=100)
df[["Amount", "AmountClipped"]].sample(10)

,Amount,AmountClipped
42951,NaN,NaN
45716,NaN,NaN
16783,NaN,NaN
42627,NaN,NaN
50744,NaN,NaN
14625,NaN,NaN
22144,NaN,NaN
42379,NaN,NaN
10488,169.892007,100.0
24707,137.668311,100.0


In [78]:
### with quantiles
df["AmountClippedQuantile"] = df["Amount"].clip(
    lower=df["Amount"].quantile(0.2),
    upper=df["Amount"].quantile(0.8)
)
df[["Amount", "AmountClippedQuantile"]].sample(10)

,Amount,AmountClippedQuantile
2886,NaN,NaN
62158,135.194043,135.194043
15124,NaN,NaN
50823,22.914443,106.869794
15659,NaN,NaN
49480,189.501765,189.501765
31152,NaN,NaN
26422,NaN,NaN
41786,NaN,NaN
71367,NaN,NaN


In [79]:
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN


In [80]:
### drop: remove columns by column label
df2 = df.drop(columns=["FirstSeenUser", "ActiveDays", "AmountKept", "AmountClipped", "AmountClippedQuantile"])
df2.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825


In [81]:
### drop: remove rows by index label
df2 = df.drop(index=[0, 1, 2, 3, 4])
df2.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
5,1,1,2024-01-01 23:09:51.956825,page_view,NaN,NaN,NaN,2024-01-01,23,2024-01-01 23:09:51.956825,1,0,0.0,NaN,NaN
6,1,1,2024-02-01 05:13:26.944169,add_to_cart,prod_4965,NaN,NaN,2024-02-01,5,2024-01-01 23:09:51.956825,0,31,0.0,NaN,NaN
7,1,1,2024-02-14 15:41:53.071258,login,NaN,NaN,NaN,2024-02-14,15,2024-01-01 23:09:51.956825,0,44,0.0,NaN,NaN
8,1,1,2024-05-20 23:13:38.011189,click,NaN,NaN,NaN,2024-05-20,23,2024-01-01 23:09:51.956825,0,140,0.0,NaN,NaN
9,1,2,2024-06-27 16:17:34.523695,page_view,NaN,NaN,NaN,2024-06-27,16,2024-01-01 23:09:51.956825,0,178,0.0,NaN,NaN


In [82]:
### drop with errors ignored
df2 = df.drop(columns='maybe_missing_column', errors='ignore')
df2.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN


In [83]:
### dropna: remove columns with all NaN values
df2 = df.dropna(axis=1, how='all')
df2.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN


In [84]:
### dropna: remove missing values
print(df.shape)

### remove rows with NaN in 'Amount' or 'EventType'
print(df.dropna(subset=['Amount', 'EventType']).shape)

(74817, 15)
(10682, 15)


In [85]:
### keep rows that have at least k non-null values
### keep rows with at least 3 non-null values
df2 = df.dropna(thresh=3)
df2.tail()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
74812,1000,10,2024-05-11 22:48:45.500117,purchase,prod_1238,186.208225,purchase,2024-05-11,22,2024-01-03 15:54:55.591340,0,129,186.208225,100.0,186.208225
74813,1000,10,2024-03-29 04:09:32.514318,logout,NaN,NaN,NaN,2024-03-29,4,2024-01-03 15:54:55.591340,0,86,0.000000,NaN,NaN
74814,1000,10,2024-02-09 02:58:56.128697,login,NaN,NaN,NaN,2024-02-09,2,2024-01-03 15:54:55.591340,0,37,0.000000,NaN,NaN
74815,1000,10,2024-04-30 16:19:48.002633,purchase,prod_2515,198.757849,purchase,2024-04-30,16,2024-01-03 15:54:55.591340,0,118,198.757849,100.0,198.757849
74816,1000,10,2024-04-01 02:19:29.148727,logout,NaN,NaN,NaN,2024-04-01,2,2024-01-03 15:54:55.591340,0,89,0.000000,NaN,NaN


In [86]:
### drop rows where all values are NaN
df2 = df.dropna(how='all')
df2.tail()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
74812,1000,10,2024-05-11 22:48:45.500117,purchase,prod_1238,186.208225,purchase,2024-05-11,22,2024-01-03 15:54:55.591340,0,129,186.208225,100.0,186.208225
74813,1000,10,2024-03-29 04:09:32.514318,logout,NaN,NaN,NaN,2024-03-29,4,2024-01-03 15:54:55.591340,0,86,0.000000,NaN,NaN
74814,1000,10,2024-02-09 02:58:56.128697,login,NaN,NaN,NaN,2024-02-09,2,2024-01-03 15:54:55.591340,0,37,0.000000,NaN,NaN
74815,1000,10,2024-04-30 16:19:48.002633,purchase,prod_2515,198.757849,purchase,2024-04-30,16,2024-01-03 15:54:55.591340,0,118,198.757849,100.0,198.757849
74816,1000,10,2024-04-01 02:19:29.148727,logout,NaN,NaN,NaN,2024-04-01,2,2024-01-03 15:54:55.591340,0,89,0.000000,NaN,NaN


In [87]:
### duplicated
dup = df.duplicated()
df[dup].head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile


In [88]:
### by subset of columns
dup = df.duplicated(subset=["UserID", "SessionID"])
df[dup].head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN
5,1,1,2024-01-01 23:09:51.956825,page_view,NaN,NaN,NaN,2024-01-01,23,2024-01-01 23:09:51.956825,1,0,0.0,NaN,NaN


In [89]:
### default: mark duplicates except for the first occurrence
df.duplicated(subset=["UserID", "SessionID"], keep="first")

0        False
1         True
2         True
3         True
4         True
         ...  
74812     True
74813     True
74814     True
74815     True
74816     True
Length: 74817, dtype: bool

In [90]:
### default: mark duplicates except for the last occurrence
df.duplicated(subset=["UserID", "SessionID"], keep="last")

0         True
1         True
2         True
3         True
4         True
         ...  
74812     True
74813     True
74814     True
74815     True
74816    False
Length: 74817, dtype: bool

In [91]:
### mark all duplicates
df.duplicated(subset=["UserID", "SessionID"], keep=False)

0        True
1        True
2        True
3        True
4        True
         ... 
74812    True
74813    True
74814    True
74815    True
74816    True
Length: 74817, dtype: bool

In [92]:
### most-frequently used cleanup: keep the last occurrence of duplicates
df.drop_duplicates(subset=["UserID", "SessionID"], keep="last").head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
8,1,1,2024-05-20 23:13:38.011189,click,NaN,NaN,NaN,2024-05-20,23,2024-01-01 23:09:51.956825,0,140,0.0,NaN,NaN
14,1,2,2024-03-06 23:01:09.517819,page_view,NaN,NaN,NaN,2024-03-06,23,2024-01-01 23:09:51.956825,0,65,0.0,NaN,NaN
23,1,3,2024-07-17 03:46:13.897763,add_to_cart,prod_6426,NaN,NaN,2024-07-17,3,2024-01-01 23:09:51.956825,0,198,0.0,NaN,NaN
33,1,4,2024-03-19 08:57:34.658875,add_to_cart,prod_5644,NaN,NaN,2024-03-19,8,2024-01-01 23:09:51.956825,0,78,0.0,NaN,NaN
42,1,5,2024-02-17 19:31:30.144156,purchase,prod_2362,35.606156,purchase,2024-02-17,19,2024-01-01 23:09:51.956825,0,47,0.0,35.606156,106.869794


### Sorting, Ranking, Ordering
- sort_values, sort_index
- nlargest, nsmallest
- rank
- reindex
- reset_index, set_index

In [93]:
df.sort_values("Amount", ascending=False).head(5)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
41716,558,3,2024-07-13 17:24:21.619401,purchase,prod_5885,499.981538,purchase,2024-07-13,17,2024-01-08 15:31:36.371406,0,187,499.981538,100.0,403.621669
41244,551,9,2024-03-31 23:28:07.302082,purchase,prod_8038,499.815255,purchase,2024-03-31,23,2024-01-04 16:33:11.953970,0,87,499.815255,100.0,403.621669
54792,733,8,2024-01-19 06:20:35.677150,purchase,prod_5885,499.800029,purchase,2024-01-19,6,2024-01-02 22:11:15.666576,0,17,499.800029,100.0,403.621669
44480,595,3,2024-02-08 01:07:46.906120,purchase,prod_2611,499.773356,purchase,2024-02-08,1,2024-01-02 10:44:35.908835,0,37,499.773356,100.0,403.621669
51077,683,9,2024-05-25 09:34:16.681479,purchase,prod_7389,499.649764,purchase,2024-05-25,9,2024-01-02 17:13:48.352839,0,144,499.649764,100.0,403.621669


In [94]:
df.sort_values(["Amount", "Timestamp"],
               ascending=[False, True],
               na_position="last").head(5)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
41716,558,3,2024-07-13 17:24:21.619401,purchase,prod_5885,499.981538,purchase,2024-07-13,17,2024-01-08 15:31:36.371406,0,187,499.981538,100.0,403.621669
41244,551,9,2024-03-31 23:28:07.302082,purchase,prod_8038,499.815255,purchase,2024-03-31,23,2024-01-04 16:33:11.953970,0,87,499.815255,100.0,403.621669
54792,733,8,2024-01-19 06:20:35.677150,purchase,prod_5885,499.800029,purchase,2024-01-19,6,2024-01-02 22:11:15.666576,0,17,499.800029,100.0,403.621669
44480,595,3,2024-02-08 01:07:46.906120,purchase,prod_2611,499.773356,purchase,2024-02-08,1,2024-01-02 10:44:35.908835,0,37,499.773356,100.0,403.621669
51077,683,9,2024-05-25 09:34:16.681479,purchase,prod_7389,499.649764,purchase,2024-05-25,9,2024-01-02 17:13:48.352839,0,144,499.649764,100.0,403.621669


In [95]:
df.sort_values(["Amount", "Timestamp"],
               ascending=[False, True],
               na_position="last",
               ignore_index=True).head(5)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
0,558,3,2024-07-13 17:24:21.619401,purchase,prod_5885,499.981538,purchase,2024-07-13,17,2024-01-08 15:31:36.371406,0,187,499.981538,100.0,403.621669
1,551,9,2024-03-31 23:28:07.302082,purchase,prod_8038,499.815255,purchase,2024-03-31,23,2024-01-04 16:33:11.953970,0,87,499.815255,100.0,403.621669
2,733,8,2024-01-19 06:20:35.677150,purchase,prod_5885,499.800029,purchase,2024-01-19,6,2024-01-02 22:11:15.666576,0,17,499.800029,100.0,403.621669
3,595,3,2024-02-08 01:07:46.906120,purchase,prod_2611,499.773356,purchase,2024-02-08,1,2024-01-02 10:44:35.908835,0,37,499.773356,100.0,403.621669
4,683,9,2024-05-25 09:34:16.681479,purchase,prod_7389,499.649764,purchase,2024-05-25,9,2024-01-02 17:13:48.352839,0,144,499.649764,100.0,403.621669


In [96]:
df.sort_values(["Amount", "Timestamp"],
               ascending=[False, True],
               na_position="first").head(5)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
13147,176,7,2024-01-01 00:01:35.004064,logout,NaN,NaN,NaN,2024-01-01,0,2024-01-01 00:01:35.004064,1,0,0.0,NaN,NaN
72919,976,4,2024-01-01 00:02:24.971658,product_view,prod_1303,NaN,NaN,2024-01-01,0,2024-01-01 00:02:24.971658,1,0,0.0,NaN,NaN
69019,925,2,2024-01-01 00:06:24.553313,login,NaN,NaN,NaN,2024-01-01,0,2024-01-01 00:06:24.553313,1,0,0.0,NaN,NaN
46632,624,1,2024-01-01 00:26:02.151406,product_view,prod_6754,NaN,NaN,2024-01-01,0,2024-01-01 00:26:02.151406,1,0,0.0,NaN,NaN
25407,341,5,2024-01-01 00:28:14.885796,login,NaN,NaN,NaN,2024-01-01,0,2024-01-01 00:28:14.885796,1,0,0.0,NaN,NaN


In [97]:
### nlargest: get top n rows by column values
### nsmallest: get bottom n rows by column values
df.nlargest(5, "Amount")

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
41716,558,3,2024-07-13 17:24:21.619401,purchase,prod_5885,499.981538,purchase,2024-07-13,17,2024-01-08 15:31:36.371406,0,187,499.981538,100.0,403.621669
41244,551,9,2024-03-31 23:28:07.302082,purchase,prod_8038,499.815255,purchase,2024-03-31,23,2024-01-04 16:33:11.953970,0,87,499.815255,100.0,403.621669
54792,733,8,2024-01-19 06:20:35.677150,purchase,prod_5885,499.800029,purchase,2024-01-19,6,2024-01-02 22:11:15.666576,0,17,499.800029,100.0,403.621669
44480,595,3,2024-02-08 01:07:46.906120,purchase,prod_2611,499.773356,purchase,2024-02-08,1,2024-01-02 10:44:35.908835,0,37,499.773356,100.0,403.621669
51077,683,9,2024-05-25 09:34:16.681479,purchase,prod_7389,499.649764,purchase,2024-05-25,9,2024-01-02 17:13:48.352839,0,144,499.649764,100.0,403.621669


In [98]:
df.nsmallest(5, "Amount")

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile
823,11,6,2024-07-08 14:49:58.100765,purchase,prod_8995,5.131843,purchase,2024-07-08,14,2024-01-01 12:27:09.150148,0,189,0.0,5.131843,106.869794
58610,784,8,2024-02-16 04:26:06.194128,purchase,prod_9314,5.224550,purchase,2024-02-16,4,2024-01-05 04:06:30.666550,0,42,0.0,5.224550,106.869794
27146,365,2,2024-03-23 00:51:40.162129,purchase,prod_2625,5.407982,purchase,2024-03-23,0,2024-01-03 18:06:54.479492,0,80,0.0,5.407982,106.869794
25656,344,7,2024-02-29 03:47:45.112983,purchase,prod_6880,5.491260,purchase,2024-02-29,3,2024-01-01 08:40:26.867813,0,59,0.0,5.491260,106.869794
57766,773,2,2024-03-28 05:56:12.425685,purchase,prod_5203,5.527694,purchase,2024-03-28,5,2024-01-02 18:42:55.372263,0,86,0.0,5.527694,106.869794


In [99]:
### rank: assign ranks to rows based on column values
df["AmountRank"] = df["Amount"].rank(ascending=False)
df[["Amount", "AmountRank"]].sample(10)

,Amount,AmountRank
31675,NaN,NaN
57360,NaN,NaN
58854,115.310853,8354.0
68449,NaN,NaN
64691,358.131565,3134.0
5065,NaN,NaN
39677,NaN,NaN
51882,NaN,NaN
37569,NaN,NaN
71574,NaN,NaN


In [100]:
### set_index
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile,AmountRank
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN,NaN


In [101]:
df.set_index("UserID").head()

,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile,AmountRank
UserID,,,,,,,,,,,,,,,
1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN,NaN
1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN,NaN
1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN,NaN
1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN,NaN
1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN,NaN


In [102]:
df.set_index(["UserID", "Timestamp"]).head()

SessionID     EventType  ProductID  Amount  \
UserID Timestamp                                                                
1      2024-07-07 18:00:26.959902          1     page_view        NaN     NaN   
       2024-03-05 22:01:00.072000          1     page_view        NaN     NaN   
       2024-03-23 22:08:10.568453          1  product_view  prod_8199     NaN   
       2024-03-12 00:32:05.495638          1      AABBCCDD  prod_4112     NaN   
       2024-02-25 22:43:01.318876          1   add_to_cart  prod_3354     NaN   

                                  Outcome        Date  Hour  \
UserID Timestamp                                              
1      2024-07-07 18:00:26.959902     NaN  2024-07-07    18   
       2024-03-05 22:01:00.072000     NaN  2024-03-05    22   
       2024-03-23 22:08:10.568453     NaN  2024-03-23    22   
       2024-03-12 00:32:05.495638     NaN  2024-03-12     0   
       2024-02-25 22:43:01.318876     NaN  2024-02-25    22   

                                                   FirstSeen  FirstSeenUser  \
UserID Timestamp                                                              
1      2024-07-07 18:00:26.959902 2024-01-01 23:09:51.956825              0   
       2024-03-05 22:01:00.072000 2024-01-01 23:09:51.956825              0   
       2024-03-23 22:08:10.568453 2024-01-01 23:09:51.956825              0   
       2024-03-12 00:32:05.495638 2024-01-01 23:09:51.956825              0   
       2024-02-25 22:43:01.318876 2024-01-01 23:09:51.956825              0   

                                   ActiveDays  AmountKept  AmountClipped  \
UserID Timestamp                                                           
1      2024-07-07 18:00:26.959902         188         0.0            NaN   
       2024-03-05 22:01:00.072000          64         0.0            NaN   
       2024-03-23 22:08:10.568453          82         0.0            NaN   
       2024-03-12 00:32:05.495638          71         0.0            NaN   
       2024-02-25 22:43:01.318876          55         0.0            NaN   

                                   AmountClippedQuantile  AmountRank  
UserID Timestamp                                                      
1      2024-07-07 18:00:26.959902                    NaN         NaN  
       2024-03-05 22:01:00.072000                    NaN         NaN  
       2024-03-23 22:08:10.568453                    NaN         NaN  
       2024-03-12 00:32:05.495638                    NaN         NaN  
       2024-02-25 22:43:01.318876                    NaN         NaN

In [103]:
df.set_index("Timestamp", drop=False).head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile,AmountRank
Timestamp,,,,,,,,,,,,,,,,
2024-07-07 18:00:26.959902,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN,NaN
2024-03-05 22:01:00.072000,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN,NaN
2024-03-23 22:08:10.568453,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN,NaN
2024-03-12 00:32:05.495638,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN,NaN
2024-02-25 22:43:01.318876,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN,NaN


### Column Creation and Transformation
- assign
- rename
- astype
- apply (row/col-wise)
- map
- replace
- transform (group-wise feature creation)
- pipe (clean pipelines)
- eval (sometimes)
- round

In [104]:
### assign: add new columns
df = df.assign(Purchase=df.Outcome == "purchase")
df[["UserID", "Outcome", "Purchase"]].sample(10)

,UserID,Outcome,Purchase
41369,553,NaN,False
36760,492,NaN,False
34608,463,NaN,False
29388,394,NaN,False
35825,480,NaN,False
10047,134,NaN,False
31487,422,NaN,False
1065,14,NaN,False
19365,260,purchase,True
31747,425,NaN,False


In [105]:
### rename: rename columns using a function
df2 = df.rename(columns=str.lower)
df2.head(3)

,userid,sessionid,timestamp,eventtype,productid,amount,outcome,date,hour,firstseen,firstseenuser,activedays,amountkept,amountclipped,amountclippedquantile,amountrank,purchase
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN,NaN,False
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN,NaN,False
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN,NaN,False


In [106]:
### rename: rename columns using a mapping dict
df2 = df2.rename(columns={"userid":"UserID", "outcome":"Outcome"})
df2.head(3)

,UserID,sessionid,timestamp,eventtype,productid,amount,Outcome,date,hour,firstseen,firstseenuser,activedays,amountkept,amountclipped,amountclippedquantile,amountrank,purchase
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN,NaN,False
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN,NaN,False
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN,NaN,False


In [107]:
### astype: cast dtypes
### astype("str")
print(df["UserID"].dtype)
df["UserID"] = df["UserID"].astype("str")
df["UserID"].dtype

int64


dtype('O')

In [108]:
### astype("boolean")
df["Purchase"] = df["Purchase"].astype("boolean")
df["Purchase"].dtype

BooleanDtype

In [109]:
### pd.to_datetime: convert to datetime dtype
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,ActiveDays,AmountKept,AmountClipped,AmountClippedQuantile,AmountRank,Purchase
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,188,0.0,NaN,NaN,NaN,False
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,0,64,0.0,NaN,NaN,NaN,False
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,0,82,0.0,NaN,NaN,NaN,False
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,0,71,0.0,NaN,NaN,NaN,False
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,0,55,0.0,NaN,NaN,NaN,False


In [110]:
### apply: (row/ col-wise) apply a function along an axis of the DataFrame
df["EventType_ProductID"] = df.apply(lambda row: f"{row.EventType}_{row.ProductID}" 
                        if pd.notna(row.ProductID) else row.EventType, 
                        axis=1)
df[["EventType", "ProductID", "EventType_ProductID"]].sample(10)

,EventType,ProductID,EventType_ProductID
43077,add_to_cart,prod_8318,add_to_cart_prod_8318
54851,click,NaN,click
13165,product_view,prod_1939,product_view_prod_1939
44379,page_view,NaN,page_view
67131,purchase,prod_8543,purchase_prod_8543
23719,add_to_cart,prod_8349,add_to_cart_prod_8349
66067,click,NaN,click
38043,purchase,prod_4748,purchase_prod_4748
42985,add_to_cart,prod_7797,add_to_cart_prod_7797
20444,logout,NaN,logout


In [111]:
df.EventType.unique()

array(['page_view', 'product_view', 'AABBCCDD', 'add_to_cart', 'login',
       'click', 'logout', 'purchase'], dtype=object)

In [112]:
### map: series value mapping, element-wise
mapping_dict = {
    "purchase": 1,
    "add_to_cart": 2,
    "product_view": 3,
    "page_view": 4,
    "login": 5,
    "logout": 6,
    "click": 7
}
df["EventTypeCode"] = df["EventType"].map(mapping_dict)
df[["EventType", "EventTypeCode"]].sample(10)

,EventType,EventTypeCode
67073,page_view,4.0
24024,page_view,4.0
34255,logout,6.0
51733,logout,6.0
36506,page_view,4.0
48846,click,7.0
47482,purchase,1.0
13479,page_view,4.0
74495,purchase,1.0
25025,click,7.0


In [113]:
### replace: series value replacement, element-wise
mapping_dict = {
    "purchase": "Purchase",
    "add_to_cart": "AddToCart",
    "product_view": "ProductView",
    "page_view": "PageView",
    "login": "Login",
    "logout": "Logout",
    "click": "Click"
}
df["EventTypeCode"] = df["EventType"].replace(mapping_dict)
df[["EventType", "EventTypeCode"]].sample(10)

,EventType,EventTypeCode
10859,logout,Logout
7715,add_to_cart,AddToCart
8128,login,Login
39758,page_view,PageView
67852,click,Click
21624,login,Login
45621,purchase,Purchase
70491,product_view,ProductView
44109,page_view,PageView
26283,logout,Logout


In [114]:
### transform: group-wise transformation
### agg: reduces rows (one row per group)
### transfrom: returns same shape as original DataFrame

### Example: z-score normalization of Amount per UserID
df["UserAmountZscore"] = df.groupby("UserID")["Amount"].transform(
    lambda x: (x - x.mean()) / x.std()
)
df[["UserID", "Amount", "UserAmountZscore"]].sample(5)

,UserID,Amount,UserAmountZscore
30251,406,NaN,NaN
50315,673,NaN,NaN
35643,477,NaN,NaN
23164,311,NaN,NaN
25724,345,NaN,NaN


In [115]:
df["UserIDAvgAmount"] = df.groupby("UserID")["Amount"].transform("mean")
df[["UserID", "Amount", "UserIDAvgAmount"]].sample(5)

,UserID,Amount,UserIDAvgAmount
18859,253,NaN,325.254309
8413,113,265.243752,246.700934
7116,96,NaN,248.782359
61355,821,NaN,259.967520
37987,508,NaN,263.949770


In [116]:
### round: round numerical columns to specified decimals
df["AmountRounded2"] = df["Amount"].round(2)
df[["Amount", "AmountRounded2"]].sample(5)

,Amount,AmountRounded2
20966,NaN,NaN
16077,NaN,NaN
28852,NaN,NaN
61444,NaN,NaN
46321,NaN,NaN


### String Manipulation (str)
- str.lower, str.upper, str.strip
- str.contains, str.match
- str.replace
- str.split, str.extract
- str.len
- str.startswith, str.endswith

In [117]:
### str methods: vectorized string operations
### str.upper(), str.lower(), str.title(), str.capitalize()
### Difference of str.title() and str.capitalize():
### str.title(): capitalize the first letter of each word
### str.capitalize(): capitalize only the first letter of the entire string

df["EventTypeCodeLower"] = df["EventTypeCode"].str.lower()
df["EventTypeCodeUpper"] = df["EventTypeCode"].str.upper()
df["EventTypeCodeTitle"] = df["EventTypeCode"].str.title()
df["EventTypeCodeCapitalized"] = df["EventTypeCode"].str.capitalize()

df[["EventTypeCode", "EventTypeCodeLower", "EventTypeCodeUpper", "EventTypeCodeTitle","EventTypeCodeCapitalized"]].sample(5)

,EventTypeCode,EventTypeCodeLower,EventTypeCodeUpper,EventTypeCodeTitle,EventTypeCodeCapitalized
38027,AddToCart,addtocart,ADDTOCART,Addtocart,Addtocart
36554,Logout,logout,LOGOUT,Logout,Logout
48874,ProductView,productview,PRODUCTVIEW,Productview,Productview
50638,PageView,pageview,PAGEVIEW,Pageview,Pageview
47566,Purchase,purchase,PURCHASE,Purchase,Purchase


In [118]:
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,...,Purchase,EventType_ProductID,EventTypeCode,UserAmountZscore,UserIDAvgAmount,AmountRounded2,EventTypeCodeLower,EventTypeCodeUpper,EventTypeCodeTitle,EventTypeCodeCapitalized
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,...,False,page_view,PageView,NaN,249.516097,NaN,pageview,PAGEVIEW,Pageview,Pageview
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,2024-03-05,22,2024-01-01 23:09:51.956825,...,False,page_view,PageView,NaN,249.516097,NaN,pageview,PAGEVIEW,Pageview,Pageview
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,2024-03-23,22,2024-01-01 23:09:51.956825,...,False,product_view_prod_8199,ProductView,NaN,249.516097,NaN,productview,PRODUCTVIEW,Productview,Productview
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,2024-03-12,0,2024-01-01 23:09:51.956825,...,False,AABBCCDD_prod_4112,AABBCCDD,NaN,249.516097,NaN,aabbccdd,AABBCCDD,Aabbccdd,Aabbccdd
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,2024-02-25,22,2024-01-01 23:09:51.956825,...,False,add_to_cart_prod_3354,AddToCart,NaN,249.516097,NaN,addtocart,ADDTOCART,Addtocart,Addtocart


In [119]:
### str.strip: remove leading and trailing whitespace
df["EventTypeCodeWithSpaces"] = "  " + df["EventTypeCode"] + "   "
df["EventTypeCodeStripped"] = df["EventTypeCodeWithSpaces"].str.strip()
print(df[["EventTypeCodeWithSpaces", "EventTypeCodeStripped"]].iloc[1, 0])
print(df[["EventTypeCodeWithSpaces", "EventTypeCodeStripped"]].iloc[1, 1])

  PageView   
PageView


In [120]:
### replace: series value replacement, element-wise
df["EventTypeCodeReplaced"] = df["EventTypeCode"].str.replace("View", "Viewed")
df[["EventTypeCode", "EventTypeCodeReplaced"]].sample(5)

,EventTypeCode,EventTypeCodeReplaced
60299,AddToCart,AddToCart
12837,Logout,Logout
12859,Purchase,Purchase
22768,Purchase,Purchase
2381,Purchase,Purchase


In [121]:
### split/ extract: split strings into multiple columns
df["Event_MainType"] = df["EventType"].str.split("_").str[0]
df["Event_SubType"] = df["EventType"].str.split("_").str[1]
df[["EventType", "Event_MainType", "Event_SubType"]].sample(5)

,EventType,Event_MainType,Event_SubType
56413,logout,logout,NaN
40646,product_view,product,view
72622,logout,logout,NaN
68581,add_to_cart,add,to
26626,login,login,NaN


In [122]:
### str.split
df[["year", "month", "day"]] = df["Date"].astype("str").str.split("-", expand=True)
df[["Date", "year", "month", "day"]].sample(5)

,Date,year,month,day
61511,2024-02-13,2024,02,13
24864,2024-02-11,2024,02,11
15408,2024-04-12,2024,04,12
48137,2024-04-29,2024,04,29
22978,2024-07-15,2024,07,15


In [123]:
### str.len
df["EventTypeLength"] = df["EventType"].str.len()
df[["EventType", "EventTypeLength"]].sample(5)

,EventType,EventTypeLength
71240,page_view,9
34695,product_view,12
11379,login,5
21861,product_view,12
45347,add_to_cart,11


In [124]:
### Prefix Checks
df[df.EventType.str.startswith("add", na=False)].sample(3)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,...,EventTypeCodeCapitalized,EventTypeCodeWithSpaces,EventTypeCodeStripped,EventTypeCodeReplaced,Event_MainType,Event_SubType,year,month,day,EventTypeLength
16535,221,9,2024-07-15 00:22:48.941587,add_to_cart,prod_6725,NaN,NaN,2024-07-15,0,2024-01-08 03:33:09.033465,...,Addtocart,AddToCart,AddToCart,AddToCart,add,to,2024,07,15,11
31497,422,7,2024-01-08 17:36:09.276769,add_to_cart,prod_8463,NaN,NaN,2024-01-08,17,2024-01-01 22:00:24.988189,...,Addtocart,AddToCart,AddToCart,AddToCart,add,to,2024,01,08,11
63877,855,4,2024-04-19 06:32:52.799478,add_to_cart,prod_7516,NaN,NaN,2024-04-19,6,2024-01-09 08:29:45.784229,...,Addtocart,AddToCart,AddToCart,AddToCart,add,to,2024,04,19,11


In [125]:
### Suffix Checks
df[df.EventType.str.endswith("view", na=False)].sample(3)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,...,EventTypeCodeCapitalized,EventTypeCodeWithSpaces,EventTypeCodeStripped,EventTypeCodeReplaced,Event_MainType,Event_SubType,year,month,day,EventTypeLength
25331,340,6,2024-03-01 06:34:45.724540,page_view,NaN,NaN,NaN,2024-03-01,6,2024-01-03 22:34:43.131245,...,Pageview,PageView,PageView,PageViewed,page,view,2024,03,01,9
1743,23,8,2024-01-30 13:58:30.836827,page_view,NaN,NaN,NaN,2024-01-30,13,2024-01-07 23:10:57.364587,...,Pageview,PageView,PageView,PageViewed,page,view,2024,01,30,9
37057,496,9,2024-03-05 11:28:18.529428,page_view,NaN,NaN,NaN,2024-03-05,11,2024-01-02 14:16:00.963624,...,Pageview,PageView,PageView,PageViewed,page,view,2024,03,05,9


### Datetime Manipulation (dt)
- pd.to_datetime
- .dt.date, .dt.normalize
- .dt.year, .dt.month, .dt.day, .dt.weekday
- .dt.floor, .dt.ceil, .dt.round
- .dt.to_period
- pd.date_range
- pd.Timedelta, pd.DateOffset

- Timezone-aware: the datetime already "knows" the timezone | `aware` -> change timezone: `tz_convert()`
- Timezone-naive: the datetime "does not know" the timezone | `naive` -> add timezone: `tz_localize()`


In [126]:
s = pd.to_datetime(df["Timestamp"], errors="coerce")
print(s.dt.tz)

### if it prints None: the datetime is timezone-naive (not timezone-aware)
### if it prints UTC or America/ New_York: the datetime is timezone-aware --- IGNORE ---

None


- First parse the datetime column, otherwise the tz func will easily fail

In [127]:
ts = pd.to_datetime(df["Timestamp"], errors="coerce")
ts

0       2024-07-07 18:00:26.959902
1       2024-03-05 22:01:00.072000
2       2024-03-23 22:08:10.568453
3       2024-03-12 00:32:05.495638
4       2024-02-25 22:43:01.318876
                   ...            
74812   2024-05-11 22:48:45.500117
74813   2024-03-29 04:09:32.514318
74814   2024-02-09 02:58:56.128697
74815   2024-04-30 16:19:48.002633
74816   2024-04-01 02:19:29.148727
Name: Timestamp, Length: 74817, dtype: datetime64[ns]

- `Native + UTC`
- If the datetime is timezone-naive, and it's UTC time, we can `add the timezone info using tz_localize('UTC')`

In [128]:
df["Timestamp_UTC"] = df["Timestamp"].dt.tz_localize("UTC")
df[["Timestamp", "Timestamp_UTC"]].head()

,Timestamp,Timestamp_UTC
0,2024-07-07 18:00:26.959902,2024-07-07 18:00:26.959902+00:00
1,2024-03-05 22:01:00.072000,2024-03-05 22:01:00.072000+00:00
2,2024-03-23 22:08:10.568453,2024-03-23 22:08:10.568453+00:00
3,2024-03-12 00:32:05.495638,2024-03-12 00:32:05.495638+00:00
4,2024-02-25 22:43:01.318876,2024-02-25 22:43:01.318876+00:00


- `Naive + NY`
- If the datetime is timezone-naive, and it's in another timezone (e.g., New York time), we can `add the timezone info using tz_localize('America/New_York')`

In [129]:
# df["Timestamp_NY"] = df["Timestamp"].dt.tz_localize("America/New_York")
# df[["Timestamp", "Timestamp_NY"]].head()

# NonExistentTimeError: 2024-03-10 02:02:55.545471 does not exist in timezone America/New_York
# This is because on March 10, 2024, at 2:00 AM, clocks in New York spring forward to 3:00 AM for Daylight Saving Time.
# Therefore, any timestamp between 2:00 AM and 2:59 AM on that date does not exist in the America/New_York timezone.

- `Convert Timezone`
- convert the timezone-aware datetime to another timezone with tz.convert()

In [130]:
# df["Timestamp_NY"] = df["Timestamp_UTC"].dt.tz_convert("America/New_York")
# df[["Timestamp", "Timestamp_UTC", "Timestamp_NY"]].head()

- Always store a canonical UTC column (Timestamp_UTC) and derive others from it with tz_convert.
- That avoids DST “nonexistent time” headaches entirely (because UTC has no gaps).

- .dt methods only work if the dtype is datetime-like or tz-aware.
- If the dtype is not datetime-like or tz-aware, you'll get an error when trying to use .dt methods

In [131]:
df["date"] = df["Timestamp"].dt.date
df[["date", "Timestamp"]].head()

,date,Timestamp
0,2024-07-07,2024-07-07 18:00:26.959902
1,2024-03-05,2024-03-05 22:01:00.072000
2,2024-03-23,2024-03-23 22:08:10.568453
3,2024-03-12,2024-03-12 00:32:05.495638
4,2024-02-25,2024-02-25 22:43:01.318876


In [132]:
df["year"] = df["Timestamp"].dt.year
df["month"] = df["Timestamp"].dt.month
df["month_name"]   = df["Timestamp"].dt.month_name()
df["day"] = df["Timestamp"].dt.day
df["weekday"] = df["Timestamp"].dt.weekday
df["weekday_name"] = df["Timestamp"].dt.day_name()

df[["Timestamp", "year", "month", "month_name", "day", "weekday_name", "weekday"]].head()

,Timestamp,year,month,month_name,day,weekday_name,weekday
0,2024-07-07 18:00:26.959902,2024,7,July,7,Sunday,6
1,2024-03-05 22:01:00.072000,2024,3,March,5,Tuesday,1
2,2024-03-23 22:08:10.568453,2024,3,March,23,Saturday,5
3,2024-03-12 00:32:05.495638,2024,3,March,12,Tuesday,1
4,2024-02-25 22:43:01.318876,2024,2,February,25,Sunday,6


- dt.floor
- dt.ceil
- dt.round

In [133]:
### .dt.floor()
df["ts_5min"] = df["Timestamp"].dt.floor("5min")
df["ts_hour"] = df["Timestamp"].dt.floor("H")
df["ts_day"]  = df["Timestamp"].dt.floor("D")
df[["Timestamp", "ts_5min", "ts_hour", "ts_day"]].sample()

,Timestamp,ts_5min,ts_hour,ts_day
25502,2024-01-07 06:56:00.710472,2024-01-07 06:55:00,2024-01-07 06:00:00,2024-01-07


In [134]:
### .dt.ceil()
df["ts_ceil_hour"]  = df["Timestamp"].dt.ceil("H")

### .dt.round()
df["ts_round_15m"]  = df["Timestamp"].dt.round("15min")
df[["Timestamp", "ts_ceil_hour", "ts_round_15m"]].sample(5)

,Timestamp,ts_ceil_hour,ts_round_15m
6791,2024-01-03 13:10:40.098984,2024-01-03 14:00:00,2024-01-03 13:15:00
23126,2024-07-17 23:11:34.486811,2024-07-18 00:00:00,2024-07-17 23:15:00
31558,2024-05-02 12:28:23.834570,2024-05-02 13:00:00,2024-05-02 12:30:00
44748,2024-04-14 21:27:43.046232,2024-04-14 22:00:00,2024-04-14 21:30:00
51871,2024-03-03 02:21:15.820816,2024-03-03 03:00:00,2024-03-03 02:15:00


In [135]:
### .dt.to_period: converts to periods like month/ quarter
df["month"] = df["Timestamp"].dt.to_period("M")
df["quarter"] = df["Timestamp"].dt.to_period("Q")

df[["Timestamp", "month", "quarter"]].head()

,Timestamp,month,quarter
0,2024-07-07 18:00:26.959902,2024-07,2024Q3
1,2024-03-05 22:01:00.072000,2024-03,2024Q1
2,2024-03-23 22:08:10.568453,2024-03,2024Q1
3,2024-03-12 00:32:05.495638,2024-03,2024Q1
4,2024-02-25 22:43:01.318876,2024-02,2024Q1


Frequency String Options
- Y, A: year
- Q: quarter
- M: month
- W: week
- D: day
- B: business day (Mon-Fri)
---
- H: hour
- T, min: minute
- S: second
- L, ms: millisecond
- U, us: microsecond
- N: nanosecond
---
- week ending day: W-MON, W-SUN, etc
- quarter/ year: Q-AMR, Q-JUN, etc

In [136]:
df.groupby(df["Timestamp"].dt.to_period("M"))["Amount"].sum()

Timestamp
2024-01    402125.529562
2024-02    376519.999302
2024-03    394057.928816
2024-04    408235.696660
2024-05    426323.724682
2024-06    402153.223718
2024-07    295156.896708
Freq: M, Name: Amount, dtype: float64

In [137]:
days = pd.date_range("2025-01-01", "2025-01-31", freq="D")
days

DatetimeIndex(['2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04',
               '2025-01-05', '2025-01-06', '2025-01-07', '2025-01-08',
               '2025-01-09', '2025-01-10', '2025-01-11', '2025-01-12',
               '2025-01-13', '2025-01-14', '2025-01-15', '2025-01-16',
               '2025-01-17', '2025-01-18', '2025-01-19', '2025-01-20',
               '2025-01-21', '2025-01-22', '2025-01-23', '2025-01-24',
               '2025-01-25', '2025-01-26', '2025-01-27', '2025-01-28',
               '2025-01-29', '2025-01-30', '2025-01-31'],
              dtype='datetime64[ns]', freq='D')

In [138]:
hours = pd.date_range("2025-01-01", periods=24, freq="H")
hours

DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 01:00:00',
               '2025-01-01 02:00:00', '2025-01-01 03:00:00',
               '2025-01-01 04:00:00', '2025-01-01 05:00:00',
               '2025-01-01 06:00:00', '2025-01-01 07:00:00',
               '2025-01-01 08:00:00', '2025-01-01 09:00:00',
               '2025-01-01 10:00:00', '2025-01-01 11:00:00',
               '2025-01-01 12:00:00', '2025-01-01 13:00:00',
               '2025-01-01 14:00:00', '2025-01-01 15:00:00',
               '2025-01-01 16:00:00', '2025-01-01 17:00:00',
               '2025-01-01 18:00:00', '2025-01-01 19:00:00',
               '2025-01-01 20:00:00', '2025-01-01 21:00:00',
               '2025-01-01 22:00:00', '2025-01-01 23:00:00'],
              dtype='datetime64[ns]', freq='H')

In [139]:
df2 = df.set_index("Timestamp").sort_index()
df2.head(4)

,UserID,SessionID,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,...,date,month_name,weekday,weekday_name,ts_5min,ts_hour,ts_day,ts_ceil_hour,ts_round_15m,quarter
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:01:35.004064,176,7,logout,NaN,NaN,NaN,2024-01-01,0,2024-01-01 00:01:35.004064,1,...,2024-01-01,January,0,Monday,2024-01-01 00:00:00,2024-01-01,2024-01-01,2024-01-01 01:00:00,2024-01-01 00:00:00,2024Q1
2024-01-01 00:02:24.971658,976,4,product_view,prod_1303,NaN,NaN,2024-01-01,0,2024-01-01 00:02:24.971658,1,...,2024-01-01,January,0,Monday,2024-01-01 00:00:00,2024-01-01,2024-01-01,2024-01-01 01:00:00,2024-01-01 00:00:00,2024Q1
2024-01-01 00:06:24.553313,925,2,login,NaN,NaN,NaN,2024-01-01,0,2024-01-01 00:06:24.553313,1,...,2024-01-01,January,0,Monday,2024-01-01 00:05:00,2024-01-01,2024-01-01,2024-01-01 01:00:00,2024-01-01 00:00:00,2024Q1
2024-01-01 00:26:02.151406,624,1,product_view,prod_6754,NaN,NaN,2024-01-01,0,2024-01-01 00:26:02.151406,1,...,2024-01-01,January,0,Monday,2024-01-01 00:25:00,2024-01-01,2024-01-01,2024-01-01 01:00:00,2024-01-01 00:30:00,2024Q1


In [140]:
print(df2.index.min(), "|", df2.index.max())

2024-01-01 00:01:35.004064 | 2024-07-24 10:13:04.983908


In [141]:
### use pd.date_range to generate time ranges
pd.date_range(df2.index.min(), df2.index.max(), freq="D")

DatetimeIndex(['2024-01-01 00:01:35.004064', '2024-01-02 00:01:35.004064',
               '2024-01-03 00:01:35.004064', '2024-01-04 00:01:35.004064',
               '2024-01-05 00:01:35.004064', '2024-01-06 00:01:35.004064',
               '2024-01-07 00:01:35.004064', '2024-01-08 00:01:35.004064',
               '2024-01-09 00:01:35.004064', '2024-01-10 00:01:35.004064',
               ...
               '2024-07-15 00:01:35.004064', '2024-07-16 00:01:35.004064',
               '2024-07-17 00:01:35.004064', '2024-07-18 00:01:35.004064',
               '2024-07-19 00:01:35.004064', '2024-07-20 00:01:35.004064',
               '2024-07-21 00:01:35.004064', '2024-07-22 00:01:35.004064',
               '2024-07-23 00:01:35.004064', '2024-07-24 00:01:35.004064'],
              dtype='datetime64[ns]', length=206, freq='D')

In [142]:
### Very common patterns: reindex to full calendar then fill
df2 = df[df.UserID == "1"]
df2 = df2.drop_duplicates(["Date"], keep="first")
df2 = df2.set_index("Timestamp").sort_index()
df2

,UserID,SessionID,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,...,date,month_name,weekday,weekday_name,ts_5min,ts_hour,ts_day,ts_ceil_hour,ts_round_15m,quarter
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 23:09:51.956825,1,1,page_view,NaN,NaN,NaN,2024-01-01,23,2024-01-01 23:09:51.956825,1,...,2024-01-01,January,0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02 00:00:00,2024-01-01 23:15:00,2024Q1
2024-01-02 00:15:51.420238,1,4,add_to_cart,prod_6144,NaN,NaN,2024-01-02,0,2024-01-01 23:09:51.956825,0,...,2024-01-02,January,1,Tuesday,2024-01-02 00:15:00,2024-01-02 00:00:00,2024-01-02,2024-01-02 01:00:00,2024-01-02 00:15:00,2024Q1
2024-01-03 23:51:05.729189,1,5,logout,NaN,NaN,NaN,2024-01-03,23,2024-01-01 23:09:51.956825,0,...,2024-01-03,January,2,Wednesday,2024-01-03 23:50:00,2024-01-03 23:00:00,2024-01-03,2024-01-04 00:00:00,2024-01-03 23:45:00,2024Q1
2024-01-04 09:25:49.705624,1,4,add_to_cart,prod_1102,NaN,NaN,2024-01-04,9,2024-01-01 23:09:51.956825,0,...,2024-01-04,January,3,Thursday,2024-01-04 09:25:00,2024-01-04 09:00:00,2024-01-04,2024-01-04 10:00:00,2024-01-04 09:30:00,2024Q1
2024-01-06 04:33:39.275154,1,9,page_view,NaN,NaN,NaN,2024-01-06,4,2024-01-01 23:09:51.956825,0,...,2024-01-06,January,5,Saturday,2024-01-06 04:30:00,2024-01-06 04:00:00,2024-01-06,2024-01-06 05:00:00,2024-01-06 04:30:00,2024Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-07 18:00:26.959902,1,1,page_view,NaN,NaN,NaN,2024-07-07,18,2024-01-01 23:09:51.956825,0,...,2024-07-07,July,6,Sunday,2024-07-07 18:00:00,2024-07-07 18:00:00,2024-07-07,2024-07-07 19:00:00,2024-07-07 18:00:00,2024Q3
2024-07-15 16:15:52.074487,1,4,purchase,prod_9886,7.677938,purchase,2024-07-15,16,2024-01-01 23:09:51.956825,0,...,2024-07-15,July,0,Monday,2024-07-15 16:15:00,2024-07-15 16:00:00,2024-07-15,2024-07-15 17:00:00,2024-07-15 16:15:00,2024Q3
2024-07-17 03:46:13.897763,1,3,add_to_cart,prod_6426,NaN,NaN,2024-07-17,3,2024-01-01 23:09:51.956825,0,...,2024-07-17,July,2,Wednesday,2024-07-17 03:45:00,2024-07-17 03:00:00,2024-07-17,2024-07-17 04:00:00,2024-07-17 03:45:00,2024Q3


In [143]:
df2 = df2.reindex(pd.date_range(df2.index.min(), df2.index.max(), freq="D")).ffill()
df2

,UserID,SessionID,EventType,ProductID,Amount,Outcome,Date,Hour,FirstSeen,FirstSeenUser,...,date,month_name,weekday,weekday_name,ts_5min,ts_hour,ts_day,ts_ceil_hour,ts_round_15m,quarter
2024-01-01 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-01-02 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-01-03 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-01-04 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-01-05 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-07-18 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-07-19 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1
2024-07-20 23:09:51.956825,1,1.0,page_view,NaN,NaN,NaN,2024-01-01,23.0,2024-01-01 23:09:51.956825,1.0,...,2024-01-01,January,0.0,Monday,2024-01-01 23:05:00,2024-01-01 23:00:00,2024-01-01,2024-01-02,2024-01-01 23:15:00,2024Q1


In [144]:
### pd.Timedelta: duration
df["Timestamp_Plus_2H"] = df["Timestamp"] + pd.Timedelta(hours=2)
df["Timestamp_Plus_7D"] = df["Timestamp"] + pd.Timedelta("7D")
df[["Timestamp", "Timestamp_Plus_2H", "Timestamp_Plus_7D"]].head()

,Timestamp,Timestamp_Plus_2H,Timestamp_Plus_7D
0,2024-07-07 18:00:26.959902,2024-07-07 20:00:26.959902,2024-07-14 18:00:26.959902
1,2024-03-05 22:01:00.072000,2024-03-06 00:01:00.072000,2024-03-12 22:01:00.072000
2,2024-03-23 22:08:10.568453,2024-03-24 00:08:10.568453,2024-03-30 22:08:10.568453
3,2024-03-12 00:32:05.495638,2024-03-12 02:32:05.495638,2024-03-19 00:32:05.495638
4,2024-02-25 22:43:01.318876,2024-02-26 00:43:01.318876,2024-03-03 22:43:01.318876


In [145]:
### .diff()
df["TimeDelta"] = df["Timestamp"].diff()
df[["Timestamp", "TimeDelta"]].head()

,Timestamp,TimeDelta
0,2024-07-07 18:00:26.959902,NaT
1,2024-03-05 22:01:00.072000,-124 days +04:00:33.112098
2,2024-03-23 22:08:10.568453,18 days 00:07:10.496453
3,2024-03-12 00:32:05.495638,-12 days +02:23:54.927185
4,2024-02-25 22:43:01.318876,-16 days +22:10:55.823238


In [146]:
### pd.DateOffset(): calendar logics such as months, business days
### Notice that pd.Timedelta(days=30) is not "one month" -> months vary
### DateOffset(months=1) is a truly month shift

df["NextMonthSameDay"] = df["Timestamp"] + pd.DateOffset(months=1)
df["PrevYear"] = df["Timestamp"] - pd.DateOffset(years=1)
df[["Timestamp", "PrevYear", "NextMonthSameDay"]].head()

,Timestamp,PrevYear,NextMonthSameDay
0,2024-07-07 18:00:26.959902,2023-07-07 18:00:26.959902,2024-08-07 18:00:26.959902
1,2024-03-05 22:01:00.072000,2023-03-05 22:01:00.072000,2024-04-05 22:01:00.072000
2,2024-03-23 22:08:10.568453,2023-03-23 22:08:10.568453,2024-04-23 22:08:10.568453
3,2024-03-12 00:32:05.495638,2023-03-12 00:32:05.495638,2024-04-12 00:32:05.495638
4,2024-02-25 22:43:01.318876,2023-02-25 22:43:01.318876,2024-03-25 22:43:01.318876


In [147]:
### Business Day Example

from pandas.tseries.offsets import BDay
df["next_bday"] = df["Timestamp"] + BDay(1)
df[["Timestamp", "next_bday"]].head()

,Timestamp,next_bday
0,2024-07-07 18:00:26.959902,2024-07-08 18:00:26.959902
1,2024-03-05 22:01:00.072000,2024-03-06 22:01:00.072000
2,2024-03-23 22:08:10.568453,2024-03-25 22:08:10.568453
3,2024-03-12 00:32:05.495638,2024-03-13 00:32:05.495638
4,2024-02-25 22:43:01.318876,2024-02-26 22:43:01.318876


### Groupby, Aggregation, and Window-ish Ops
- groupby
- agg (named aggregation)
- size, count, sum, mean, median, min, max, std
- nunique, first, last
- value_counts (within group via groupby(...).value_counts())
- transform
- apply (with caution)
- cumcount, cumsum, cummax, cummin
- shift, diff
- pct_change

In [148]:
cols = ["UserID", "SessionID", "Timestamp", "EventType", "ProductID", "Amount", "Outcome"]
df = df[cols].copy()

In [149]:
### Total events per user
df.groupby(["UserID"]).size()

UserID
1       82
10      81
100     76
1000    82
101     76
        ..
995     75
996     75
997     78
998     72
999     75
Length: 1000, dtype: int64

In [150]:
### Events per (User, Session)
df.groupby(["UserID", "SessionID"]).size()

UserID  SessionID
1       1             9
        2             6
        3             9
        4            10
        5             9
                     ..
999     6             5
        7            10
        8             8
        9            10
        10            9
Length: 10000, dtype: int64

In [151]:
### Named aggregation (best practice for clean columns)
out = (df.groupby("UserID")
         .agg(
             n_events_NA=("EventType", "size"), # including NA
             n_events=("EventType", "count"), # not including NA
             n_sessions=("SessionID", "nunique"),
             total_amt=("Amount", "sum"),
             avg_amt=("Amount", "mean"),
             first_ts=("Timestamp", "min"),
             last_ts=("Timestamp", "max"),
         )
         .reset_index())
out.head()

,UserID,n_events_NA,n_events,n_sessions,total_amt,avg_amt,first_ts,last_ts
0,1,82,82,10,1996.128780,249.516097,2024-01-01 23:09:51.956825,2024-07-22 20:10:14.181302
1,10,81,81,10,3567.334590,198.185255,2024-01-01 08:01:45.649799,2024-07-23 22:50:17.279206
2,100,76,76,10,3591.513958,239.434264,2024-01-01 18:49:13.031574,2024-07-24 10:13:04.983908
3,1000,82,82,10,3056.475836,218.319703,2024-01-03 15:54:55.591340,2024-07-21 05:48:07.728611
4,101,76,76,10,3057.296506,203.819767,2024-01-02 23:53:55.015078,2024-07-24 07:37:22.603563


- size, count, sum, mean, median, min, max, std

In [152]:
# df.groupby("UserID").size()
# df.groupby("UserID")["Amount"].count()
# df.groupby("UserID")["Amount"].sum()
# df.groupby("UserID")["Amount"].mean()
# df.groupby("UserID")["Amount"].std()

- nunique, first, last

In [153]:
# unique event type per user
df.groupby("UserID")["EventType"].nunique().reset_index().sort_values("UserID", ascending=True)

,UserID,EventType
0,1,8
1,10,7
2,100,7
3,1000,7
4,101,7
...,...,...
995,995,7
996,996,7
997,997,7
998,998,7


In [154]:
# first/last product id per session, depending on the row order
df_sorted = df.sort_values(["SessionID", "ProductID"])

out = df_sorted.groupby("SessionID")["ProductID"].first()
out2 = df_sorted.groupby("SessionID")["ProductID"].last()

In [155]:
out.head()

SessionID
1    prod_1000
2    prod_1001
3    prod_1001
4    prod_1006
5    prod_1007
Name: ProductID, dtype: object

In [156]:
out2.head()

SessionID
1    prod_9997
2    prod_9994
3    prod_9999
4    prod_9995
5    prod_9999
Name: ProductID, dtype: object

In [157]:
### value_counts within group
### top event types by each user
counts = (df.groupby("UserID")["EventType"]
            .value_counts()
            .rename("n_EventType")
            .reset_index())
counts

,UserID,EventType,n_EventType
0,1,add_to_cart,22
1,1,page_view,15
2,1,product_view,14
3,1,login,11
4,1,purchase,8
...,...,...,...
6996,999,click,11
6997,999,logout,11
6998,999,page_view,10
6999,999,add_to_cart,9


In [158]:
### top 1 event type by each user
top1 = (counts.sort_values(["UserID", "n_EventType"], ascending=[True, False])
              .groupby("UserID")
              .head(1))
top1

,UserID,EventType,n_EventType
0,1,add_to_cart,22
8,10,purchase,18
15,100,product_view,16
22,1000,logout,15
29,101,purchase,15
...,...,...,...
6966,995,login,13
6973,996,click,12
6980,997,product_view,15
6987,998,product_view,17


In [159]:
### transform: group-wise features
df["UserAvgAmount"] = df.groupby("UserID")["Amount"].transform("mean")
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,UserAvgAmount
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,249.516097
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,249.516097
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,249.516097
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,249.516097
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,249.516097


In [160]:
### Z-score within the User
g = df.groupby("UserID")["Amount"]
df["AmountZscore"] = (df["Amount"] - g.transform("mean")) / g.transform("std")
df.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,UserAvgAmount,AmountZscore
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN,249.516097,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,249.516097,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN,249.516097,NaN
3,1,1,2024-03-12 00:32:05.495638,AABBCCDD,prod_4112,NaN,NaN,249.516097,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,249.516097,NaN


In [161]:
### Rank with sessions
df_sorted = df.sort_values(["SessionID", "Timestamp"])
df_sorted["EventRank"] = df_sorted.groupby("SessionID").cumcount() + 1
df_sorted.tail()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,UserAvgAmount,AmountZscore,EventRank
7551,101,10,2024-07-24 07:37:22.603563,purchase,prod_4254,82.10062,purchase,203.819767,-0.953084,7427
66618,891,10,2024-07-24 08:26:18.040987,click,NaN,NaN,NaN,261.554034,NaN,7428
51459,688,10,2024-07-24 09:21:08.889919,logout,NaN,NaN,NaN,250.022975,NaN,7429
1457,19,10,2024-07-24 09:24:35.846976,page_view,NaN,NaN,NaN,301.640693,NaN,7430
58486,782,10,2024-07-24 09:41:11.623478,click,NaN,NaN,NaN,218.001706,NaN,7431


In [162]:
### cumcount, cumsum, cummax, cummin (running features)

# df_sorted = df.sort_values(["UserID", "Timestamp"])
# df_sorted["CumAmount"] = df_sorted.groupby("UserID").cumcount()
# df_sorted["CumsumAmount"] = df_sorted.groupby("UserID")["Amount"].cumsum()
# df_sorted["CummaxAmount"] = df_sorted.groupby("UserID")["Amount"].cummax()

In [163]:
### shift

df_sorted = df[df["UserID"] == "1"].sort_values(["UserID", "SessionID", "Timestamp"])
df_sorted.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome,UserAvgAmount,AmountZscore
5,1,1,2024-01-01 23:09:51.956825,page_view,NaN,NaN,NaN,249.516097,NaN
6,1,1,2024-02-01 05:13:26.944169,add_to_cart,prod_4965,NaN,NaN,249.516097,NaN
7,1,1,2024-02-14 15:41:53.071258,login,NaN,NaN,NaN,249.516097,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN,249.516097,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN,249.516097,NaN


In [164]:
df_sorted["PrevTimestamp"] = df_sorted.groupby("SessionID")["Timestamp"].shift(1)
df_sorted["SecGap"] = (df_sorted["Timestamp"] - df_sorted["PrevTimestamp"]).dt.total_seconds()

df_sorted[["UserID", "SessionID", "Timestamp", "PrevTimestamp", "SecGap"]].head()

,UserID,SessionID,Timestamp,PrevTimestamp,SecGap
5,1,1,2024-01-01 23:09:51.956825,NaT,NaN
6,1,1,2024-02-01 05:13:26.944169,2024-01-01 23:09:51.956825,2.613815e+06
7,1,1,2024-02-14 15:41:53.071258,2024-02-01 05:13:26.944169,1.160906e+06
4,1,1,2024-02-25 22:43:01.318876,2024-02-14 15:41:53.071258,9.756682e+05
1,1,1,2024-03-05 22:01:00.072000,2024-02-25 22:43:01.318876,7.750788e+05


In [165]:
### diff

df_sorted["TimestampDiff"] = df_sorted["Timestamp"].diff().dt.total_seconds()
df_sorted[["UserID", "SessionID", "Timestamp", "TimestampDiff", "SecGap"]].head()

,UserID,SessionID,Timestamp,TimestampDiff,SecGap
5,1,1,2024-01-01 23:09:51.956825,NaN,NaN
6,1,1,2024-02-01 05:13:26.944169,2.613815e+06,2.613815e+06
7,1,1,2024-02-14 15:41:53.071258,1.160906e+06,1.160906e+06
4,1,1,2024-02-25 22:43:01.318876,9.756682e+05,9.756682e+05
1,1,1,2024-03-05 22:01:00.072000,7.750788e+05,7.750788e+05


In [166]:
### pct_change

df2 = (df.groupby(["UserID", "SessionID"])["Amount"].sum()
           .rename("SessionTotalAmount")
           .reset_index()
           .sort_values(["UserID", "SessionID", "SessionTotalAmount"]))
df2["pct_change"] = df2["SessionTotalAmount"].pct_change()
df2.head()
 

,UserID,SessionID,SessionTotalAmount,pct_change
0,1,1,0.000000,NaN
1,1,2,0.000000,NaN
2,1,3,72.913619,inf
3,1,4,7.677938,-0.894698
4,1,5,998.570616,129.057139


### Reshaping data (winde <-> long)
- `pivot, pivot_table`: long->wide
- `melt`: wide->long (unpivot columns into variables + values)
- `stack`: columns->rows, requires MultiIndex, common after unstack
- `unstack`: rows->columns, requires MultiIndex, common after groupby
- `crosstab`: quick count table

- pivot, pivot_table: Long->Wide table

In [167]:
session_wide = (
    df.pivot_table(
        index=["UserID", "SessionID"], # group rows by each user-session pair, the output will have one row per session
        columns="EventType", # take every unique value in EventType, and make them separate columns 
        values="Timestamp",
        aggfunc="count",  # inside each (UserID, SessionID, EventType) bucket, count how many rows there are
        fill_value=0 # If a session never had a certain event type, you’d normally get NaN. This turns those into 0
    )
    .reset_index()
)
session_wide.head()

EventType,UserID,SessionID,AABBCCDD,add_to_cart,click,login,logout,page_view,product_view,purchase
0,1,1,1,2,1,1,0,3,1,0
1,1,2,0,0,1,0,1,3,1,0
2,1,3,0,3,1,2,0,1,1,1
3,1,4,0,4,0,2,0,2,1,1
4,1,5,0,0,1,1,1,1,1,4


In [168]:
prod_funnel = (
    df.dropna(subset=["ProductID"])
      .assign(flag=1)
      .pivot_table(
          index=["UserID","SessionID","ProductID"],
          columns="EventType",
          values="flag",
          aggfunc="max",
          fill_value=0
      )
      .reset_index()
)

prod_funnel.head()

EventType,UserID,SessionID,ProductID,AABBCCDD,add_to_cart,product_view,purchase
0,1,1,prod_3354,0,1,0,0
1,1,1,prod_4112,1,0,0,0
2,1,1,prod_4965,0,1,0,0
3,1,1,prod_8199,0,0,1,0
4,1,2,prod_2683,0,0,1,0


- crosstab: counts how many times categories occur across combinations of columns

In [169]:
pd.crosstab(df["EventType"], df["Outcome"])          # EventType x Outcome


Outcome,purchase
EventType,
purchase,10682


In [170]:
pd.crosstab(df["UserID"], df["EventType"])          # User x EventType counts

EventType,AABBCCDD,add_to_cart,click,login,logout,page_view,product_view,purchase
UserID,,,,,,,,
1,1,22,5,11,6,15,14,8
10,0,7,9,11,12,14,10,18
100,0,9,6,7,14,9,16,15
1000,0,11,12,11,15,5,14,14
101,0,10,10,11,10,11,9,15
...,...,...,...,...,...,...,...,...
995,0,9,10,13,12,11,11,9
996,0,10,12,8,9,12,12,12
997,0,7,12,12,11,14,15,7


In [171]:
pd.crosstab(df["SessionID"], df["EventType"])       # Session x EventType counts

EventType,AABBCCDD,add_to_cart,click,login,logout,page_view,product_view,purchase
SessionID,,,,,,,,
1,1,1063,1080,1008,1085,1094,1063,1026
2,0,1083,997,1016,1124,1065,1107,1043
3,0,1071,1068,1085,1083,1090,1085,1016
4,0,1055,1074,1083,1059,1057,1042,1091
5,0,1063,1101,1111,1026,1084,1038,1058
6,0,1091,1068,1038,1000,1067,1085,1082
7,0,1074,1064,1051,1061,1057,1087,1138
8,0,1086,1074,1080,1083,1078,1054,1125
9,0,1083,1040,1070,1107,1111,1085,1052


- melt: Wide->Long table


In [172]:
prod_funnel.head()

EventType,UserID,SessionID,ProductID,AABBCCDD,add_to_cart,product_view,purchase
0,1,1,prod_3354,0,1,0,0
1,1,1,prod_4112,1,0,0,0
2,1,1,prod_4965,0,1,0,0
3,1,1,prod_8199,0,0,1,0
4,1,2,prod_2683,0,0,1,0


In [173]:
long_again = prod_funnel.melt(
    id_vars=["UserID", "SessionID", "ProductID"],
    var_name="EventType",
    value_name="Count"
)

long_again.sample(10)

,UserID,SessionID,ProductID,EventType,Count
32205,100,5,prod_6113,add_to_cart,0
79417,526,1,prod_5707,product_view,0
47320,526,5,prod_7684,add_to_cart,0
30135,943,5,prod_6699,AABBCCDD,0
2630,172,2,prod_8874,AABBCCDD,0
45233,467,5,prod_5170,add_to_cart,0
101192,235,10,prod_7213,purchase,0
106588,387,7,prod_4862,purchase,1
27881,881,1,prod_8680,AABBCCDD,0
70958,290,4,prod_8904,product_view,0


- stack/ unstack

In [174]:
g = df.groupby(["UserID", "SessionID", "EventType"]).size()
g.head()

UserID  SessionID  EventType  
1       1          AABBCCDD       1
                   add_to_cart    2
                   click          1
                   login          1
                   page_view      3
dtype: int64

In [175]:
### same idea as pivot, but via MultiIndex
wide = g.unstack("EventType", fill_value=0).reset_index()
wide.head()

EventType,UserID,SessionID,AABBCCDD,add_to_cart,click,login,logout,page_view,product_view,purchase
0,1,1,1,2,1,1,0,3,1,0
1,1,2,0,0,1,0,1,3,1,0
2,1,3,0,3,1,2,0,1,1,1
3,1,4,0,4,0,2,0,2,1,1
4,1,5,0,0,1,1,1,1,1,4


In [176]:
back_to_long = wide.set_index(["UserID","SessionID"]).stack().reset_index(name="Count")
back_to_long.head()

,UserID,SessionID,EventType,Count
0,1,1,AABBCCDD,1
1,1,1,add_to_cart,2
2,1,1,click,1
3,1,1,login,1
4,1,1,logout,0


### Joining, Merging, Concatenation
- `merge` (left/ right/ inner/ outer, validate=): SQL join on columns
- `join`: merge on index (handy after set_index)
- `concat`: stack data vertically/ horizontally (no key matching)
- `merge_asof` (time-series “closest prior” joins): time-based "latest prior/ nearest" matching
- `combine_first`: fill missing values from another source

In [177]:
### merge()

# out = left.merge(right, on="UserID", how="inner")   # only matches
# out = left.merge(right, on="UserID", how="left")    # keep all left rows
# out = left.merge(right, on="UserID", how="right")    # keep all right rows
# out = left.merge(right, on="UserID", how="outer")   # keep all rows from both

- join on different column names

In [178]:
# out = left.merge(right, left_on="user_id", right_on="UserID", how="left")

- multiple keys

In [179]:
# out = left.merge(right, on=["UserID", "SessionID"], how="left")

- validate: catch data bugs

In [180]:
# out = left.merge(
#     right, 
#     on="UserID", 
#     how="left", 
#     validate="one_to_many"
# )

- "validate" is a safety check on the key uniqueness before pandas actually does the merge.
- If the data doesn’t match the relationship you claim, pandas raises a MergeError.

`This enforces`
- left: each UserID appears at most once (unique key)
- right: a UserID can appear multiple times

So one row on the left can match many rows on the right → output can have more rows than left (row “duplication” of left rows).

In [181]:
# both sides unique per key

# out = left.merge(right, on="UserID", how="left", validate="one_to_one")

`This enforces`
- left: UserID is unique
- right: UserID is also unique

So each left row matches 0 or 1 right row → output row count will be exactly len(left) (for a left join).

- join() vs. merge()
- `merge` joins on columns, `join` joins on index (unless you pass on=).
- join(): index-based merge shortcut

In [182]:
# out = left.set_index("UserID").join(
#     right.set_index("UserID"),
#     how="left",
#     rsuffix="_r"
# ).reset_index()

- `concat`: Used to append rows or append columns

In [183]:
# ### stack rows: 

# # same columns, more rows
# out = pd.concat([df_jan, df_feb], axis=0, ignore_index=True)

# # same index, more columns
# out = pd.concat([features_a, features_b], axis=1)

- `merge_asof`: Time-series "closest prior" join

- Use when you want to match each row to the nearest earlier (or nearest) timestamp.
- Classic: join trades to last quote, or events to the latest feature snapshot.

`Requirements`
- Both sides sorted by TimeoutError
- time keys are datetime

In [184]:
# events = events.sort_values("Timestamp")
# snap   = snap.sort_values("Timestamp")

# out = pd.merge_asof(
#     events,
#     snap,
#     on="Timestamp",
#     by="UserID",          # optional: also match within each user
#     direction="backward", # "backward" (last prior), "forward", or "nearest"
#     tolerance=pd.Timedelta("5min")  # optional max gap
# )

- `combine-first`: fill missing values from another series/ df


In [185]:
# df["Amount_final"] = df["Amount_primary"].combine_first(df["Amount_backup"])

### Missing Data Handling
- isna, notna
- fillna (scalar / method like ffill/bfill)
- ffill, bfill
- interpolate (common in time series)
- dropna

- `isna()`
- `notna()`

In [186]:
data = pd.DataFrame({
    "UserID": [1, 2, 3, 4],
    "Amount": [10.0, np.nan, 30.0, np.nan],
    "Outcome": ["success", None, "fail", None],
})
data

,UserID,Amount,Outcome
0,1,10.0,success
1,2,NaN,None
2,3,30.0,fail
3,4,NaN,None


In [187]:
# boolean mask DataFrame
data.isna()

,UserID,Amount,Outcome
0,False,False,False
1,False,True,True
2,False,False,False
3,False,True,True


In [188]:
data["Outcome"].isna()

0    False
1     True
2    False
3     True
Name: Outcome, dtype: bool

In [189]:
data[data["Amount"].notna()]

,UserID,Amount,Outcome
0,1,10.0,success
2,3,30.0,fail


- `fillna()`

In [190]:
data["AmountFilled"] = data["Amount"].fillna(0)
data["OutcomeFilled"] = data["Outcome"].fillna("unknown")
data.head()

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled
0,1,10.0,success,10.0,success
1,2,NaN,None,0.0,unknown
2,3,30.0,fail,30.0,fail
3,4,NaN,None,0.0,unknown


- `fillna()` with dict (different columns, different values)

In [191]:
data2 = data.fillna({"Amount": 0, "Outcome": "unknown"})
data2.head()

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled
0,1,10.0,success,10.0,success
1,2,0.0,unknown,0.0,unknown
2,3,30.0,fail,30.0,fail
3,4,0.0,unknown,0.0,unknown


- forward fill: `ffill()`
- backward fill: `bfill()`

In [192]:
data_fill = data.sort_values("UserID").copy()
data_fill["Outcome_ffill"] = data_fill["Outcome"].ffill()
data_fill["Outcome_bfill"] = data_fill["Outcome"].bfill()
data_fill.head()

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled,Outcome_ffill,Outcome_bfill
0,1,10.0,success,10.0,success,success,success
1,2,NaN,None,0.0,unknown,success,fail
2,3,30.0,fail,30.0,fail,fail,fail
3,4,NaN,None,0.0,unknown,fail,None


In [193]:
### Equivalent
data_fill["Outcome_ffill_2"] = data_fill["Outcome"].fillna(method="ffill")
data_fill.head()

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled,Outcome_ffill,Outcome_bfill,Outcome_ffill_2
0,1,10.0,success,10.0,success,success,success,success
1,2,NaN,None,0.0,unknown,success,fail,success
2,3,30.0,fail,30.0,fail,fail,fail,fail
3,4,NaN,None,0.0,unknown,fail,None,fail


- `interpolate()`: time series
- good for numeric series when smoothing fills

In [194]:
ts = pd.DataFrame(
    {"y": [1.0, np.nan, np.nan, 4.0]},
    index=pd.to_datetime(["2024-01-01", "2024-01-02", "2024-01-03", "2024-01-04"])
)
ts.head()

,y
2024-01-01,1.0
2024-01-02,NaN
2024-01-03,NaN
2024-01-04,4.0


In [195]:
ts["y_linear"] = ts["y"].interpolate()  # 2.0, 3.0 filled linearly
ts.head()

,y,y_linear
2024-01-01,1.0,1.0
2024-01-02,NaN,2.0
2024-01-03,NaN,3.0
2024-01-04,4.0,4.0


- `dropna()`: drop rows with any missing
- `dropna(axis=1)`: drop columns that have any missing


In [196]:
data

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled
0,1,10.0,success,10.0,success
1,2,NaN,None,0.0,unknown
2,3,30.0,fail,30.0,fail
3,4,NaN,None,0.0,unknown


In [197]:
data_drop_any = data.dropna()
data_drop_any

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled
0,1,10.0,success,10.0,success
2,3,30.0,fail,30.0,fail


In [198]:
df_drop_thresh = data.dropna(thresh=2)  # keep rows with >=2 non-NA values
df_drop_thresh

,UserID,Amount,Outcome,AmountFilled,OutcomeFilled
0,1,10.0,success,10.0,success
1,2,NaN,None,0.0,unknown
2,3,30.0,fail,30.0,fail
3,4,NaN,None,0.0,unknown


In [199]:
df_drop_cols = data.dropna(axis=1)
df_drop_cols

,UserID,AmountFilled,OutcomeFilled
0,1,10.0,success
1,2,0.0,unknown
2,3,30.0,fail
3,4,0.0,unknown


### Binning & Discretization
- cut
- qcut

### Numeric/ Vectorized Operations (often with Numpy)
- np.where, np.select
- Series.clip
- Series.abs
- Series.round
- arithmetic ops (+, -, *, /)
- Series.div, mul, add, sub
- Series.to_numpy

### Rolling, Expanding, EWM windows
- rolling (...): mean/sum/std/min/max/quantile
- expanding(...): mean/sum
- ewm(...): mean (exponential moving average)

### Category + encoding helpers

- astype("category")
- cat.codes, cat.categories
- get_dummies

### Performance / engineering essentials
- copy (avoid view bugs)
- astype (downcast types)
- select_dtypes
- memory_usage(deep=True)
- DataFrame.equals (sanity checks)
- Series.map vs merge (lookup tables)
- pd.options.mode.chained_assignment (understand warnings)